# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
T. Henning  ->  T. Henning  |  ['T. Henning']


Arxiv has 75 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.27032


extracting tarball to tmp_2510.27032...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.27086
extracting tarball to tmp_2510.27086...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.27468


...

 done.


Found 105 bibliographic references in tmp_2510.27505/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.27591


extracting tarball to tmp_2510.27591... done.


T. Henning  ->  T. Henning  |  ['T. Henning']


Found 68 bibliographic references in tmp_2510.27591/lPup.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.27468-b31b1b.svg)](https://arxiv.org/abs/2510.27468) | **Dust and Water in V883 Ori: Relics of a Retreating Snowline**  |
|| <mark>Y. Wang</mark>, et al. |
|*Appeared on*| *2025-11-03*|
|*Comments*| *Accepted for publication in A&A Letter*|
|**Abstract**|            V883 Ori is an FU-Orionis-type outburst system characterized by a shoulder at 50-70 au in its ALMA band 6 and 7 intensity profiles. Previously, this feature was attributed to dust pile-up from pebble disintegration at the water snowline. However, recent multi-wavelength observations show continuity in the spectral index across the expected snowline region, disfavoring abrupt changes in grain properties. Moreover, extended water emission is detected beyond 80 au, pointing to a snowline further out. This Letter aims to explain both features with a model in which the snowline is receding. We construct a 2D disk model that solves the cooling and subsequent vapor recondensation during the post-outburst dimming phase. Our results show that both the intensity shoulder and the extended water emission are natural relics of a retreating snowline: the shoulder arises from excess surface density generated by vapor recondensation at the moving condensation front, while the outer water vapor reservoir persists due to the long recondensation timescales of $10^{2}-10^{3}$ yr at the disk atmosphere. As V883 Ori continues to fade, we predict that the intensity shoulder will migrate inward by an observationally significant amount of 10 au over about 25 years.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.27505-b31b1b.svg)](https://arxiv.org/abs/2510.27505) | **Euclid: Systematic uncertainties from the halo mass conversion on galaxy cluster number count data analyses**  |
|| T. Gayoux, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-03*|
|*Comments*| *20 pages, 11 figures, submitted to A&A*|
|**Abstract**|            The large catalogues of galaxy clusters expected from the Euclid survey will enable cosmological analyses of cluster number counts that require accurate cosmological model predictions. One possibility is to use parametric fits calibrated against $N$-body simulations, that capture the cosmological parameter dependence of the halo mass function. Several studies have shown that this can be obtained through a calibration against haloes with spherical masses defined at the virial overdensity. In contrast, if different mass definitions are used for the HMF and the scaling relation, a mapping between them is required. Here, we investigate the impact of such a mapping on the cosmological parameter constraints inferred from galaxy cluster number counts. Using synthetic data from $N$-body simulations, we show that the standard approach, which relies on assuming a concentration-mass relation, can introduce significant systematic bias. In particular, depending on the mass definition and the relation assumed, this can lead to biased constraints at more than 2$\sigma$ level. In contrast, we find that in all the cases we have considered, the mass conversion based on the halo sparsity statistics result in a systematic bias smaller than the statistical error.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.27591-b31b1b.svg)](https://arxiv.org/abs/2510.27591) | **Multi-band infrared imaging reveals dusty spiral arcs around the binary B[e] star 3 Puppis**  |
|| M. Abello, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-11-03*|
|*Comments*| *19 pages, 11 figures, 11 tables*|
|**Abstract**|            3 Puppis is the brightest known B[e] star. Recent work classifies this A-type object as a supergiant, yet the impact of its binarity on the circumstellar environment (CE) remains hard to characterize. To resolve its dusty region at 5-10 mas, we obtained mid-IR interferometric observations with VLTI/MATISSE over 3-12 {\mu}m. Because the (u,v) coverage supports imaging, we introduce a statistical interferometric-imaging workflow based on MiRA to generate averaged images: this systematic approach enables the selection of an optimal set of reconstructions, improving the robustness and fidelity of the recovered features. We also use SPARCO, an independent tool well suited to bright central objects embedded in fainter extended emission. Images from both tools in the L, M, and N bands agree and reveal an asymmetric, elongated feature ~17 mas (~10 au at 631 pc) southeast of the star with ~20% density contrast. A second northwest asymmetry and a skewed inner rim are detected. Simple geometric modelling, guided by the MATISSE images, constrains the morphology, location, and flux of the CE and its asymmetries. The images are consistent with earlier VLTI measurements but expose a more complex CE with large-scale clumps in the southeast and northwest parts of the disc. Hydrodynamic modelling indicates that tidal spiral-wake perturbations from the central binary, dynamically excited at Lindblad resonances in the circumbinary disc, best explain the radial extent and curvature of the elongated structures seen in all bands.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.27032-b31b1b.svg)](https://arxiv.org/abs/2510.27032) | **Photometric Redshifts in JWST Deep Fields: A Pixel-Based Alternative with DeepDISC**  |
|| G. Merz, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-03*|
|*Comments*| *17 pages, 16 figures, catalog available at this https URL code available at this https URL*|
|**Abstract**|            Photo-z algorithms that utilize SED template fitting have matured, and are widely adopted for use on high-redshift near-infrared data that provides a unique window into the early universe. Alternative photo-z methods have been developed, largely within the context of low-redshift optical surveys. Machine learning based approaches have gained footing in this regime, including those that utilize raw pixel information instead of aperture photometry. However, the efficacy of image-based algorithms on high-redshift, near-infrared data remains underexplored. Here, we test the performance of Detection, Instance Segmentation and Classification with Deep Learning (DeepDISC) on photometric redshift estimation with NIRCam images from the JWST Advanced Deep Extragalactic Survey (JADES) program. DeepDISC is designed to produce probabilistic photometric redshift estimates directly from images, after detecting and deblending sources in a scene. Using NIRCam-only images and a compiled catalog of spectroscopic redshifts, we show that DeepDISC produces reliable photo-zs and uncertainties comparable to those estimated from template fitting using HST+JWST filters; DeepDISC even outperforms template fitting (lower scatter/fewer outliers) when the input photometric filters are matched. Compared with template fitting, DeepDISC does not require measured photometry from images, and can produce a catalog of 94000 photo-zs in ~4 minutes on a single NVIDIA A40 GPU. While current spectroscopic training samples are small and incomplete in color-magnitude space, this work demonstrates the potential of DeepDISC for increasingly larger image volumes and spectroscopic samples from ongoing and future programs. We discuss the impact of the training data on applications to broader samples and produce a catalog of photo-zs for all JADES DR2 photometric sources in the GOOD-S field, with quality flags indicating caveats.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.27086-b31b1b.svg)](https://arxiv.org/abs/2510.27086) | **Conditional variational autoencoders for cosmological model discrimination and anomaly detection in cosmic microwave background power spectra**  |
|| T.-Y. Sun, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-03*|
|*Comments*| *19 pages, 13 figures*|
|**Abstract**|            The cosmic microwave background power spectra are a primary window into the early universe. However, achieving interpretable, likelihood-compatible compression and fast inference under weak model assumptions remains challenging. We propose a parameter-conditioned variational autoencoder (CVAE) that aligns a data-driven latent representation with cosmological parameters while remaining compatible with standard likelihood analyses. The model achieves high-fidelity compression of the $D_\ell^{TT}$, $D_\ell^{EE}$, and $D_\ell^{TE}$ spectra into just 5 latent dimensions, with reconstruction accuracy exceeding $99.9\%$ within Planck uncertainties. It reliably reconstructs spectra for beyond-$\Lambda$CDM scenarios, even under parameter extrapolation, and enables rapid inference, reducing the computation time from $\sim$40 hours to $\sim$2 minutes while maintaining posterior consistency. The learned latent space demonstrates a physically meaningful structure, capturing a distributed representation that mirrors known cosmological parameters and their degeneracies. Moreover, it supports highly effective unsupervised discrimination among cosmological models, achieving performance competitive with supervised approaches. Overall, this physics-informed CVAE enables anomaly detection beyond $\Lambda$CDM and points to physically meaningful directions for refinement.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.27468/./fig/dimming_curve_revise.png', 'tmp_2510.27468/./fig/sigma_time_revise_45_85.png', 'tmp_2510.27468/./fig/bestfit_intens_revise_45_85.png']
copying  tmp_2510.27468/./fig/dimming_curve_revise.png to _build/html/
copying  tmp_2510.27468/./fig/sigma_time_revise_45_85.png to _build/html/
copying  tmp_2510.27468/./fig/bestfit_intens_revise_45_85.png to _build/html/
exported in  _build/html/2510.27468.md
    + _build/html/tmp_2510.27468/./fig/dimming_curve_revise.png
    + _build/html/tmp_2510.27468/./fig/sigma_time_revise_45_85.png
    + _build/html/tmp_2510.27468/./fig/bestfit_intens_revise_45_85.png
found figures ['tmp_2510.27505/./figures/Flagship_all_200c_zcuts.png', 'tmp_2510.27505/./figures/Flagship_all_500c_zcuts.png', 'tmp_2510.27505/./figures/euclid_all_200c.png', 'tmp_2510.27505/./figures/euclid_all_500c.png', 'tmp_2510.27505/./figures/side_by_side_comparison_vir_Flagship.png']
copying  tmp_2510.27505/./figures/Flagship_all_200c_zcuts.png to _build/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{çc}[1]{\textcolor{orange}{[\textit{CWO: \small #1}]}}$
$\newcommand{\coadd}[1]{\textcolor{orange}{#1}\xspace}$
$\newcommand{\coch}[2]{\textcolor{mygray}{\sout{#1}}{\textcolor{orange}{#2}}}$
$\newcommand{\corem}[1]{\textcolor{mygray}{\sout{#1}}}$
$\newcommand{\cim}[1]{\textcolor{orange}{[\textit{\starchris\star}:\textbf{\small #1}}]}$
$\newcommand{\?}{\textcolor{orange}{^{\bm{??}}}}$
$\newcommand{\md}{\textbf{}}$
$\newcommand{\mdd}{\textbf{}}$
$\newcommandinecolor{mygray}{gray}{0.6}$
$\newcommand{\ywc}[1]{[\textcolor[RGB]{128,0,128}{\small YW: \textit{#1}}]}$
$\newcommand{\ywadd}[1]{\textcolor[RGB]{128,0,128}{#1}}$
$\newcommand{\ywch}[2]{\textcolor{mygray}{\sout{#1}}{\textcolor[RGB]{139,0,139}{{#2}}}}$
$\newcommand{\ywrem}[1]{\textcolor{mygray}{\sout{#1}}}$
$\newcommand{\App}[1]{Appendix~\ref{sec:#1}}$
$\newcommand{\app}[1]{App.~\ref{sec:#1}}$
$\newcommand{\Se}[1]{Section~\ref{sec:#1}}$
$\newcommand{\se}[1]{Sect.~\ref{sec:#1}}$
$\newcommand{\Fg}[1]{Figure~\ref{fig:#1}}$
$\newcommand{\fg}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{\fgs}[2]{Figs.~\ref{fig:#1} and \ref{fig:#2}}$
$\newcommand{\tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\Tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\eq}[1]{equation~(\ref{eq:#1})}$
$\newcommand{\Eq}[1]{Equation~(\ref{eq:#1})}$
$\newcommand{\Eqs}[2]{Equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqs}[2]{equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqto}[2]{equations~(\ref{eq:#1})--(\ref{eq:#2})}$
$\newcommand\gas{\mathrm{g}}$
$\newcommand\dust{\mathrm{d}}$
$\newcommand\parti{\mathrm{p}}$
$\newcommand\diff{\mathrm{d}}$
$\iftrue$
$\newcommand{\ywc}[1] $
$\newcommand{\ywrem}[1]{\xspace}$
$\newcommand{\ywch}[2]{#2}$
$\newcommand{\ywadd}[1]{#1\xspace}$
$\newcommand{\cim}[1] $
$\newcommand{\?} $
$\newcommand{\corem}[1]{\xspace}$
$\newcommand{çc}[1]{\xspace}$
$\newcommand{\coch}[2]{#2}$
$\newcommand{\coadd}[1]{#1\xspace}$
$\newcommand{\md} $
$\newcommand{\mdd} $
$\fi$
$\begin{document}$
$   \title{Dust and Water in V883 Ori: Relics of a Retreating Snowline}$
$   \author{$
$            Y.~Wang (\begin{CJK*}{UTF8}{gbsn}王雨\end{CJK*})$
$          \inst{1} \orcidlink{0009-0004-2217-4439}$
$          \and$
$          C.W.~Ormel\inst{1} \orcidlink{0000-0003-4672-8411}$
$          \and$
$          H.-C. Jiang (\begin{CJK*}{UTF8}{gbsn}蒋昊昌\end{CJK*})\inst{2} \orcidlink{0000-0003-2948-5614}$
$          \and$
$          S.~Krijt\inst{3} \orcidlink{0000-0002-3291-6887}$
$          \and$
$          A.~Houge\inst{4} \orcidlink{0000-0001-8790-9011}$
$          \and$
$          E.~Macías\inst{5} \orcidlink{0000-0003-1283-6262}$
$          }$
$        \institute{Department of Astronomy, Tsinghua University, 30 Shuangqing Rd, Haidian DS, 100084 Beijing, China\ 	\email{wang-y21@mails.tsinghua.edu.cn}$
$        \and$
$            Max Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany$
$        \and$
$            Department of Physics and Astronomy, University of Exeter, Exeter, EX4 4QL, UK$
$         \and$
$            Center for Star and Planet Formation, GLOBE Institute, University of Copenhagen, Øster Voldgade 5-7, DK-1350 Copenhagen, Denmark$
$        \and$
$            European Southern Observatory, Karl-Schwarzschild-Str 2, 85748 Garching, Germany}$
$   \date{Received September 15, 1996; accepted March 16, 1997}$
$\abstract{$
$    V883 Ori is an FU-Orionis-type outburst system characterized by a shoulder at 50-70 au in its ALMA band 6 and 7 intensity profiles. Previously, this feature was attributed to dust pile-up from pebble disintegration at the water snowline. However, recent multi-wavelength observations show continuity in the spectral index across the expected snowline region, disfavoring abrupt changes in grain properties. Moreover, extended water emission is detected beyond 80 au, pointing to a snowline further out. This Letter aims to explain both features with a model in which the snowline is receding.$
$    We construct a 2D disk model that solves the cooling and subsequent vapor recondensation during the post-outburst dimming phase.$
$    Our results show that both the intensity shoulder and the extended water emission are natural relics of a retreating snowline: the shoulder arises from excess surface density generated by vapor recondensation at the moving condensation front, while the outer water vapor reservoir persists due to the long recondensation timescales of 10^{2}-10^{3} yr at the disk atmosphere.$
$    As V883 Ori continues to fade, we predict that the intensity shoulder will migrate inward by an observationally significant amount of 10 au over about \md{25} years.}$
$   \keywords{Protoplanetary disks --$
$                Planets and satellites: formation --$
$                Stars: protostars$
$               }$
$   \maketitle$
$\section{Introduction}$
$In protoplanetary disks, snowlines mark the locations where volatile ice sublimates. This process dictates volatile distributions and alters grain properties in the disk (e.g., \citealt{OebergEtal2023}).$
$In this context, outburst systems are ideal laboratories for studying the effect of snowlines. Due to an abrupt increase in accretion, these young stellar objects brighten by many orders of magnitude, heating up the disk for tens to hundreds of years and enabling large-scale sublimation of ices (e.g., \citealt{FischerEtal2023}). In particular, the snowline is pushed to \sim10--100 au, bringing it within reach of facilities such as the Atacama Large Millimeter/submillimeter Array (ALMA).$
$V883 Ori is one of the best-studied FU Orionis outburst systems. \citet{CiezaEtal2016} identified a "dark annulus" in ALMA band 6 continuum,  \md{suggesting} that its water snowline lies at \approx40 au. This shoulder-like feature in the intensity profile was initially attributed to the accumulation of small grains inside the snowline, produced by the \md{disintegration} of dust grains after ice sublimation \citep{SaitoSirono2011, AumatellWurm2011}.$
$However, multi-wavelength analysis has revealed \md{continuity in the} spectral index across the intensity shoulder, implying no abrupt change of the grain physical properties by ice sublimation \citep{HougeKrijt2023,HougeEtal2024}.$
$The exact snowline location is further complicated by the detection of H_{2}^{18}O \citep{TobinEtal2023} and indirect tracers like methanol or HCO^{+} \citep{van'tHoffEtal2018, LeemkerEtal2021}, which suggest a snowline location {\gtrsim}80 \mathrm{au}.$
$In this Letter, we demonstrate that these observations can be consistently explained by a model in which the snowline retreats \md{along} with the decline of bolometric luminosity. At the \md{inward-}moving snowline front, the intensity shoulder \md{arises from} the \md{increase in} surface density of solids due to ice recondensation, while in the outer disk the sluggish recondensation of vapor preserves a relic of a previously hot state.$
$\section{Model}$
$\label{sec:model}$
$We set up the disk at the outburst active phase, when the bolometric luminosity reaches its peak value (see \se{dim_curve}), assuming the dust-vapor mixture is in hydrostatic balance and thermal equilibrium. The main focus of this work is to model the subsequent dimming phase, when L_{\mathrm{bol}} declines and vapor recondenses onto grain surfaces. A key assumption is that dimming proceeds rapidly ({\sim}200~\mathrm{yr}; see \se{dim_curve}) so that dynamic processes (gas advection, pebble drift, and diffusion) can be neglected.$
$\subsection{Disk structure}$
$\label{sec:disk_model}$
$The gas surface density profile is taken as a power law$
$\begin{equation}$
$\label{eq:sigma_gas}$
$    \Sigma_{\gas} (r) = \Sigma_{c} \left( \frac{r}{r_{c}}\right)^{-\gamma}.$
$\end{equation}$
$Following the fit of dust thermal emission by \citet{CiezaEtal2018}, \md{we take r_{c} = 31 \mathrm{au} and \Sigma_{c}=160~\mathrm{g~cm^{-2}}, while \gamma is a treated as a parameter given its poor constraint \citep{van'tHoffEtal2018}}. The surface density of pebbles is taken as:$
$\begin{equation}$
$\label{eq:sigma_peb}$
$    \Sigma_{\parti}= Z_{\mathrm{peb}} \Sigma_{\gas}(r_{c}) \left( \frac{r}{r_{c}}\right)^{-\gamma_{\mathrm{peb}}},$
$\end{equation}$
$where Z_\mathrm{peb} is the metallicity at the reference position.$
$Beyond the snowline, pebbles are \md{assumed} 50\% \md{refractory} and 50\% water ice by mass, \mdd{with the material densities of constituent species following DSHARP \citep{BirnstielEtal2018}}.$
$Following \citet{WangEtal2025} we construct a 2D model in the disk's R{-}z plane, where a standard viscous disk with \alpha=10^{-3} \citep{ShakuraSunyaev1973} is assumed. For solids, we adopt an MRN size distribution \citep{MathisEtal1977}, which is appropriate when dust growth is limited by fragmentation \citep{BirnstielEtal2011}. The size distribution is characterized by an upper bound s_{\max}, corresponding to a midplane Stokes number \mathrm{St}_{0}, a lower bound of s_{\min}=0.1 \mu\mathrm{m}, and is assumed unchanged by the outburst \citep{HougeEtal2024}.$
$Pebbles of different sizes are vertically distributed according to their scale height H_{\mathrm{p},i}(H_\mathrm{g}, \mathrm{St}_{i,0},\alpha), where H_{\gas} is the gas pressure scale height and \mathrm{St}_{i,0} the Stokes number of pebble i at the midplane (see \app{dust_bins}).$
$The vapor condensation rate (\mathcal{R}_{\mathrm{cond}}) is determined by its saturation condition and the total surface area of particles (e.g., \citealt{RosJohansen2013, SchoonenbergOrmel2017}). Specifically, the vapor density at any point in the disk reduces at a rate,$
$\begin{equation}$
$\label{eq:limiter}$
$    -\frac{\diff \rho_\mathrm{vap}}{\diff t}$
$    = \mathcal{R}_{\rm cond}$
$    = \langle \pi s^{2}_{\parti} n_\parti \rangle v_{\mathrm{th,vap}} \rho_{\mathrm{vap}} \left(1 - \frac{P_{\mathrm{eq}}}{P_{\mathrm{vap}}}\right).$
$\end{equation}$
$where \langle \pi s^{2}_{\parti} n_\parti \rangle is the surface area of pebbles per unit volume (\app{dust_bins}, \eq{surface_area}).$
$\subsection{Luminosity during outburst dimming phase}$
$\label{sec:dim_curve}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/dimming_curve_revise.pdf}$
$    \caption{Evolution of the bolometric luminosity of V883 Ori during the outburst dimming phase. The symbols denote measurements from different studies, \md{with horizontal bars showing the time range of adopted data}. The blue band represents the adopted luminosity curve, which carries an uncertainty of {\sim}40~\mathrm{yrs} due to the uncertainties in estimating L_{\mathrm{bol}}.}$
$    \label{fig:dimming_curve}$
$\end{figure}$
$We model the dimming of V883 Ori following the published luminosities, accounting for observational uncertainties arising from dust extinction and instrumental saturation (see \app{L_bol_uncertain}). Specifically, the time-dependent bolometric luminosity reads,$
$\begin{equation}$
$    \label{eq:Lfunc}$
$    L_{\mathrm{bol}} (t) = L_{\star} + (L_{\mathrm{pk}} - L_{\star}) \exp\left[-0.5\left(\frac{t-t_{\mathrm{beg}}}{50~\mathrm{yr}}\right)^{2} \right],$
$\end{equation}$
$where L_{\star} is the stellar luminosity (6 L_{\odot}; \citealt{CiezaEtal2016}). We adopt a peak luminosity of L_{\mathrm{pk}}=4 000 L_{\odot} (comparable to that used in \citealt{LeemkerEtal2021}), which pushes the water snowline to 80--100~au. The parameter t_{\mathrm{beg}} indicates the year when the outburst begins to fade.  As shown in \fg{dimming_curve}, choosing t_{\mathrm{beg}}=\md{1880} A.D. makes the dimming curve consistent with the lower limit of the estimated L_{\mathrm{bol}} of V883 Ori (400 L_{\odot}, \citealt{StromStrom1993}; and 218 L_{\odot}, \citealt{FurlanEtal2016}). By shifting t_{\mathrm{beg}} forward by 40 \mathrm{yr} (A.D.~1920), the curve reaches the upper limit (647 L_{\odot}) given by \citet{LiuEtal2022}. Recently, \citet{CarvalhoEtal2025} fit the spectrum at 0.4-4.2 \mu \mathrm{m} and obtained an accretion luminosity of 458 L_{\odot}, lying well within the assumed range. Through \eq{Lfunc}, the model output is only a function of t'=t-t_\mathrm{beg}, the time since L_{\mathrm{bol}} starts to decline.$
$\subsection{Temperature model}$
$\label{sec:T_model}$
$We adopt a two-stream radiation transfer (2sRT) method to calculate the temperature (e.g., \citealt{Hubeny1990}).$
$The heating sources at each grid cell are due to irradiation (q_{\mathrm{irr}}), viscous dissipation (q_\mathrm{vis}) and latent heat exchange (q_{\mathrm{latent}}).$
$We compute q_{\mathrm{irr}} from the bolometric luminosity (\eq{Lfunc}), while the viscous heating rate follows from the \alpha-disk viscosity \md{(q_{\mathrm{vis}}=\frac{9}{4} \rho \alpha c_{\mathrm{s}}^{2} \Omega)} and the latent heating rate is proportional to the \md{condensation} rate (\eq{limiter}, see \citealt{WangEtal2025}).$
$In the outburst active phase, the disk adjusts to a temperature and density structure determined by the bolometric luminosity L_\mathrm{pk}, assuming hydrostatic balance.$
$In the dimming phase, following \citet{FlockEtal2017}, we let the disk adapt to a new equilibrium temperature obtained by the 2sRT (T_{\mathrm{eq}}) over a thermal relexation timescale, \mathrm{d} T /\mathrm{d}t = (T_{\mathrm{eq}} - T) /t_{\mathrm{relax}} (see \app{t_relax} for the calculation of t_{\mathrm{relax}}). The Rosseland mean opacity per unit gas mass \kappa_{\mathrm{R}} -- a key quantity that determines t_{\mathrm{relax}} -- is a free model parameter. In our simulation (see \se{results}), due to the low metallicity and disk viscosity, the disk temperature is \md{determined} by irradiation, with q_\mathrm{vis} and q_{\mathrm{latent}} contributing little.$
$\subsection{Fitting ALMA observations}$
$\label{sec:fit_alma}$
$The intensity shoulder in V883 Ori is identified in both ALMA band 6 (0.038", project code: 2015.1.00350.S, PI: Lucas Cieza) and band 7 (0.028", project code: 2016.1.00728.S, PI: Lucas Cieza) high-resolution continuum images.$
$\md{We converted the continuum images into radial profiles by azimuthally averaging the data within concentric annuli of radial width 1/5 of the respective beam size. To fit the continuum profiles,} we generate synthetic intensity profiles in both bands by conducting 2sRT with \md{profiles for the} dust temperature and density taken from the simulation results. The 2sRT is performed in plane-parallel manner along the disk's vertical direction, with the optical depth corrected for disk inclination i=38.3^{\circ} \citep{CiezaEtal2016}. Pebble opacities for different sizes and ice fractions are computed with \texttt{optool} \citep{DominikEtal2021}, adopting the DSHARP optical constants \citep{BirnstielEtal2018}. \md{After being convolved with the beam size, the synthetic intensity profiles are compared with the continuum data at 45–85 au, where the intensity shoulder lies}.$
$\citet{TobinEtal2023} measured the column density of water isotope H_{2}^{18}O in band 5 (0.126"). Given the limited data sensitivity, we fit only the mean column density between 80–120 au in our simulations to the observed value, assuming an isotopic ratio ^{18}O/^{16}O = 1/560 \citep{WilsonRood1994}. In reality, the observed water vapor abundance may be influenced by processes beyond simple recondensation (\se{water_emission}). Therefore, we introduce an additional free parameter f_{\mathrm{cond}} such that, - \dust \rho_{\mathrm{vap}} / \dust t = f_{\mathrm{cond}} \mathcal{R}_{\mathrm{cond}}.$
$To incorporate observational constraints from both continuum and water emission, we apply a \md{log-normal} likelihood for each dataset, \md{and} weight each likelihood with \md{the number of beams covered by the area of interest} (see \eqs{likelihood_i}{likelihood}).$
$\section{Results}$
$\label{sec:results}$
$\newcommand{\arraystretch}{1.2}$
$\begin{table}[]$
$    \centering$
$    \caption{Overview of MCMC parameters and posterior values \md{([16,50,84]-th percentiles)} for both the retreating snowline and static snowline model.}$
$    \begin{tabular}{c|l|c|c}$
$        \hline$
$         & Prior & \multicolumn{2}{c}{\makecell{Posterior \ retreating / static}} \         \hline$
$       \log Z_{\mathrm{peb}} & [-3.5, -2] & -2.89^{+0.05}_{-0.05} & -2.84^{+0.007}_{-0.007}  \         \gamma & [-2.0, -1.0] & -1.31^{+0.22}_{-0.34}  & -1.99^{+0.005}_{-0.002} \         \gamma_{\mathrm{peb}} & [-1.5, -0.5] & -0.97^{+0.10}_{-0.10} & -0.50^{+0.004}_{-0.008} \         \log St_{0} & [-3, -1] & -1.81^{+0.13}_{-0.12} & -2.71^{+0.016}_{-0.010} \         t^{\prime} (yr) & [60, 170] & 120.8^{+9.3}_{-9.0} & -  \         \log f_{\mathrm{cond}} & [-0.5, 1.5] &  0.53^{+0.09}_{-0.08} & -\         \log \kappa_{\mathrm{R}} & [0, 1.3] & 0.67^{+0.17}_{-0.15} & 0.006^{+0.001}_{-0.000} \         \hline$
$        L_{\mathrm{bol}} (L_{\odot}) & [0, 978] & - & 489.9^{+6.4}_{-6.0} \         \hline$
$    \end{tabular}$
$    \tablefoot{Z_{\mathrm{peb}}: pebble metallicity (Eq.\ref{eq:sigma_peb}); \gamma, \gamma_{\mathrm{peb}}: gas and pebble surface density index (Eq.\ref{eq:sigma_gas},\ref{eq:sigma_peb}); St_{0}: Stokes number of maximum grain size (\se{disk_model}, \ref{sec:dust_bins}); t^{\prime}: time since the outburst dimming (Eq.\ref{eq:Lfunc});$
$    f_{\mathrm{cond}}: condensation rate correction factor (\se{fit_alma}); \kappa_{\mathrm{R}}: Rosseland-mean opacity (\se{T_model}, \ref{sec:t_relax}); L_{\mathrm{bol}}: bolometric luminosity, for static model (\se{static_model}).}$
$    \label{tab:mcmc_paras}$
$\end{table}$
$We conduct Markov chain Monte Carlo (MCMC) analysis with \textit{emcee} \citep{Foreman-MackeyEtal2013} to obtain the posterior distribution (see \se{MCMC_result}). The \md{parameter's} posterior values are given in \tb{mcmc_paras}.$
$\subsection{Emergence of intensity shoulder}$
$\label{sec:intensity_shoulder}$
$The best-fit results are shown in \fg{obv_compare}, where the synthetic intensity profiles agree well with the dual-band data, especially capturing the intensity shoulder feature at 50-70 au (gray-shaded region). To understand this, we plot the time evolution of the corresponding surface density profiles in \fg{sigma_time}. As the disk cools during the outburst dimming phase, vapor gradually recondenses on pebbles, creating a bump in surface density at the condensation front, which appears in continuum as the intensity shoulder. Matching the observed position of the shoulder requires the snowline to retreat to \approx60 au at the observational epoch (t'=121 \mathrm{yr}). This places a stringent constraint on the thermal relaxation time, or IR-opacity \kappa_{\mathrm{R}}, of the disk. If the disk had cooled more rapidly (lower \kappa_R), the snowline would already have retreated inside 50 au, shifting the shoulder much closer to the star, within the expected observational time.$
$Simultaneously fitting the dual-band continuum yields \mathrm{St_{0}} \approx 0.01, corresponding to a maximum grain size of {\sim}centimeter. The need of having cm-sized particles in the disk follows from the low spectral indices at 50-70 au (\alpha=2.5-3.0), a result consistently found across ALMA band 3-7 in V883 Ori by \citet{HougeEtal2024}.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/sigma_time_revise_45_85.pdf}$
$    \caption{Time evolution of the surface density profiles of the best-fit model. Silicate surface density remains constant with time. The thick line denotes the best-fit t^{\prime}=121 yr.}$
$    \label{fig:sigma_time}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/bestfit_intens_revise_45_85.pdf}$
$    \caption{Comparison of the synthetic intensity profiles with ALMA continuum. Crosses show ALMA data adopted from \citet{HougeEtal2024}, spaced according to the beam sizes; shaded regions indicate the rms error. Solid lines represent the best-fit retreating snowline model, while dotted lines correspond to the best-fit static model. Enhanced emission towards the inner regions arises likely from intense viscous heating \citep{AlarconEtal2024}, which is not accounted for in the model.}$
$    \label{fig:obv_compare}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/H2O18_time_revise_45_85.pdf}$
$    \caption{Average column density of H_{2}^{18}O at 80-120 au under different condensation rate (f_{\mathrm{cond}}). The vertical grey band indicates the time range when observing the intensity shoulder. The dashed \md{black} line denotes the average column density derived from ALMA band 5 data by \citet{TobinEtal2023} with 1\sigma uncertainty shaded in grey. \md{Dotted grey line denotes the best-fit value in static snowline model.}}$
$    \label{fig:water_sigma}$
$\end{figure}$
$\subsection{Extended water emission}$
$\label{sec:water_emission}$
$The mean column density of H_{2}^{18}O at 80-120 au, as estimated from our models assuming optically thin emission, is presented in \fg{water_sigma}.$
$In all simulated cases, the water abundance drops sharply within the first 110 years, and transitions to a slowly-evolving stage afterwards. This is due to different condensation timescales in the disk's midplane and the upper layers (see \app{2D_structure}). After the snowline retreats, vapor in the midplane recondenses rapidly. Conversely, in the disk's upper layer, where pebble densities are much lower, the recondensation time can reach {\sim}10^2{-}10^3 \mathrm{yr} \citep{VisserEtal2015,RabEtal2017}.$
$Therefore, due to the lag caused by the finite recondensation time, the snowline inferred from molecular emission would consistently appear further away than its actual position during the outburst dimming phase.$
$Our modeling shows that an enhanced condensation rate (f_\mathrm{cond}\approx4) is required to match the inferred water vapor abundance, which may indicate a shallower size distribution and/or porous grains \citep{Birnstiel2024}, both of which increase the total dust surface area at the disk atmosphere.$
$In addition, the destruction of water by photodissociation, heterogeneous nucleation and different outburst duration may also influence the observed gas-phase water abundance \citep{FischerEtal2023,RosJohansen2024,Romero-MirzaEtal2024i}.$
$\subsection{The need for a retreating snowline model}$
$\label{sec:static_model}$
$A retreating snowline naturally explains the distant intensity shoulder and the extended water emission, relics of the earlier outburst.$
$In contrast, a static snowline model fails to explain the observations (\fg{obv_compare} and \fg{corner_static}).$
$To illustrate this, we performed another MCMC simulation, where the disk structure is set by a constant L_{\mathrm{bol}} (treated as a free parameter) rather than evolving with the luminosity curve. In other words, the temperature and vapor equilibrium instantly adapt to the stellar luminosity.$
$\md{Because of the instantaneous adjustment, the static model fails to maintain a sufficient amount of vapor in the outer disk: the best-fit water number density is \approx7 \sigma away from the observed value (\fg{water_sigma}). Furthermore, in the static model, the snowline is located at \approx30 au, failing to reproduce the shoulder feature at 50-70 au in the observed intensity profile.}$
$\section{Conclusions and discussions}$
$\label{sec:conclusion}$
$We investigated V883 Ori's continuum and water line emission by a retreating snowline model.$
$The key findings are:$
$   \begin{enumerate}$
$       \item The dimming stage of a stellar outburst results in a receding water snowline. Vapor condensation at the snowline front creates a bump in surface density, marking the transition from icy to ice-free pebbles.$
$       \item Vapor in the disk upper layer takes {\sim}10^{2}{-}10^3 \mathrm{yr} to recondense, leading to substantial amounts of water vapor in the outer disk, even after the snowline has retreated inwards. The observed H_2^{18}O outside the current snowline region (50-70 au) is a leftover of a much hotter past phase.$
$       \item The retreating snowline model simultaneously reproduces the intensity shoulder feature in ALMA band 6/7 continuum and the extended water emission. In contrast, a static model fails to maintain the snowline at 50-70 au under the present-day luminosity.$
$\end{enumerate}$
$In our modeling, both the disk cooling time (t_{\mathrm{relax}}), which controls the snowline retreat speed, and the mm opacity, which sets the continuum emission, depends on dust opacities. Thus, the thermal history of outburst systems constrains dust properties. For example, the retreating snowline model requires \kappa_{\mathrm{R}} \approx 4.6~\mathrm{cm^{2}~g^{-1}}, whereas a self-consistent calculation from the DSHARP opacity yields \kappa_{\mathrm{R}} \approx 0.4~\mathrm{cm^{2}~g^{-1}}.$
$This mismatch implies excess IR opacity relative to millimeter, possibly arising from a non-MRN size distribution, carbon-rich small grains \citep{WoitkeEtal2016}, or porous dust \citep{ZhangEtal2023}.$
$\md{Under the assumed dimming curve of luminosity}, our model predicts that the intensity shoulder will move by 10 au over \md{25} years (\fg{sigma_time}), a trend testable with future observations. With ALMA’s growing sample of outburst systems at high resolution, we expect the intensity shoulder to emerge as an ubiquitous feature of large-scale water phase transitions in disks.$
$\begin{acknowledgements}$
$    The authors thank for useful discussion with Vardan Elbakyan, Shangjia Zhang, Hanpu Liu. This work is supported by the National Natural Science Foundation of China under grant Nos. 12233004 and 12473065.$
$\end{acknowledgements}$
$\bibliographystyle{aa}$
$\bibliography{ads}$
$\appendix$
$\section{Bolometric luminosity measurements}$
$\label{sec:L_bol_uncertain}$
$The onset of the outburst in V883 Ori was not directly observed. But it likely starts before A.D.~1888, when the reflection nebula IC 430 was observed to be illuminated by the outburst \citep{Pickering1890}, as argued by \citet{StromStrom1993}.$
$\md{By fitting the spectral energy distribution, the bolometric luminosity of V883 Ori was estimated to be 400~L_{\odot} \citep{StromStrom1993} in the 1980s} \citep{AllenEtal1975,NakajimaEtal1986} and had significantly decreased to 218~L_{\odot} \citep{FurlanEtal2016, ConnelleyReipurth2018} in about 20 years. Given the typical month-to-year rise time of FUors (e.g., \citealt{FischerEtal2023}), it is very likely that V883 Ori has been in the dimming phase since A.D.~1888.$
$However, estimating the bolometric luminosity of V883 Ori is fraught with substantial uncertainty. As a young stellar object, it suffers severe extinction and reddening at near-infrared wavelengths from dusty proto-stellar envelopes. In the mid-infrared, where extinction is less severe, observations are hindered by heavy saturation in modern surveys like WISE \citep{ContrerasPenaEtal2020}. \citet{LiuEtal2022} derived a much higher bolometric luminosity of {\approx}647 L_{\odot} by accounting for the reddening.$
$\md{Unlike most works adopting discrete photometric data, recently, \citet{CarvalhoEtal2025} fits the medium resolution spectrophotometry within a range of 0.4-2.5 \mu\mathrm{m} and found excellent match to even broader spectrum across 0.4-4.2 \mu\mathrm{m}. They yielded an accretion luminosity of 458 L_{\odot}, which lies well inside the assumed luminosity curve (\fg{dimming_curve}).}$
$\section{Dust model}$
$\label{sec:dust_bins}$
$At each radius, we model the pebble size distribution by constructing several size bins spanning radii from s_{\max} and s_{\min}.$
$For small pebbles (e.g., s_{\parti}{<}0.1~\mathrm{cm}), logarithmic spacing is used, whereas for larger ones, the size bins are more evenly spaced.$
$The pebble surface density for each size bin (\Sigma_{\parti,i}) is then obtained by dividing \Sigma_{\parti} into these size bins according to the MRN distribution \diff N_{\parti} / \diff s_{\parti} \propto s_{\parti}^{-3.5}.$
$For each size bin i, its Stokes number (\mathrm{St}_{i}) is obtained by scaling the pebble size at the bin center (s_{\parti,i}) with s_{\max}, assuming that the pebbles follow the Epstein drag law: i.e., \mathrm{St}_{i,0}= \mathrm{St}_{0} ~s_{\parti,i}/s_{\max}. Then the 2D pebble density distribution is obtained by adopting normal distribution in the vertical direction:$
$\begin{equation}$
$\label{eq:rho_peb}$
$  \rho_{\parti,i} (z) = \rho_{\parti,0} \exp\left[ -\frac{1}{2} \left( \frac{z}{H_{\parti,i}}\right)^2 \right].$
$\end{equation}$
$Here H_{\parti,i} is the scale height of pebbles in the size bin i, following \citet{DubrulleEtal1995}:$
$\begin{equation}$
$\label{eq:H_peb}$
$    H_{\parti,i} (z)= H_{\gas} \left[ 1 + \frac{\mathrm{St}_{i,0}}{\alpha}\left(\frac{2 H_{\gas}^{2}}{z^{2}}\right)\left(\exp \left(\frac{z^{2}}{2 H_{\gas}^{2}}\right)-1\right) \right]^{-1/2}.$
$\end{equation}$
$In \eq{H_peb}, the vertical variation of Stokes number due to gas stratification is accounted for, \mathrm{St}_{i}(z) = \mathrm{St}_{i,0} \rho_{\gas}(z)/\rho_{\gas}(0)  \approx \mathrm{St}_{i,0} \exp \left(z^{2}/ 2 H_{\gas}^{2}\right). The gas scale height H_{\gas} is determined in the outburst active phase, after iterating between the gas density distribution, assuming hydrostatic balance, and the temperature structure calculated from 2sRT.$
$Given \Sigma_{\parti,i}, the midplane pebble density \rho_{\parti,0} can be obtained by integrating the vertical density distribution following \eq{rho_peb}.$
$Recently, \citet{HougeEtal2024} preformed dust evolution modeling and demonstrated that, to match the observed spectral index in V883 Ori, pebble sizes should not evolve appreciably during the swift outburst event. Therefore, we maintain a constant pebble size distribution throughout the outburst dimming phase.$
$Given the size distribution, the total surface area of pebbles per unit volume can be obtained by summing over all size bins,$
$\begin{equation}$
$    \label{eq:surface_area}$
$    \langle \pi s^{2}_{\parti} n_\parti \rangle = \pi\sum_{i} s_{\parti,i}^{2} n_{\parti,i},$
$\end{equation}$
$which is used to compute the vapor recondensation rate \mathcal{R}_{\mathrm{cond}}. The number density of particles in each size bin is given as n_{\parti,i} = \rho_{\parti,i}/m_{\parti,i}, where m_{\parti,i} = 4\pi \rho_{\bullet}  s_{\parti,i}^{3}/3 is the particle mass. We adopt \rho_{\bullet}=1.5~\mathrm{g~cm}^{-3} as the internal density of pebbles, assuming a compact ice-silicate mixture with half ice in mass (e.g., \citealt{SchoonenbergOrmel2017}).$
$\section{Thermal relaxation timescale}$
$\label{sec:t_relax}$
$Following \citet{FlockEtal2017}, we combine the optically thin and optically thick limits to determine the thermal relaxation time:$
$\begin{equation}$
$\label{eq:t_cool}$
$    t_{\mathrm{relax}} = t_{\mathrm{thin}} + t_{\mathrm{thick}} = \frac{l_{\mathrm{mfp}}^{2}}{3D_{\mathrm{rad}}} + \frac{H_{\mathrm{phot}}^{2}}{D_{\mathrm{rad}}}$
$\end{equation}$
$where l_{\mathrm{mfp}} = 1/(\kappa_{\mathrm{R}} \rho_{\gas}) is the mean free path of photons, D_{\mathrm{rad}} = 16 \sigma_{\mathrm{b}} T^{3} / (3\kappa$
$_{\mathrm{R}} \rho_{\gas}^{2} C_{\mathrm{v}}) is the radiation diffusion coefficient and H_{\mathrm{phot}} is the height of disk photosphere (\tau_{\mathrm{R}}=2/3), as measured from the simulation. Here C_{\mathrm{v}} = 8.2\times10^{7}~\mathrm{erg~g^{-1}~K^{-1}} is the specific heat capacity of the He-H_{2} mixture (assuming 71\% H_{2} and ideal gas) and \kappa_{\mathrm{R}} is the Rosseland mean opacity per gas mass, which is a free parameter.$
$\section{2D structure of retreating snowline}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.9\textwidth]{fig/2D_structure_revise_45_85.png}$
$    \caption{Disk temperature, vapor condensation timescale (t_{\mathrm{cond}}) and number density of water (n_{\mathrm{H_{2}\mathrm{O}}}) during the outburst dimming phase of the best-fit model. Here t^{\prime}=t-t_{\mathrm{beg}} is the time lapse since the bolometric luminosity started to decline. The value of  L_{\mathrm{bol}} corresponding to this time is labeled.$
$    In the top row, the range of 140-150 K is shaded in white to highlight the snowline region.$
$    At t^{\prime}=121~\mathrm{yr} -- the present epoch -- the emission surfaces (\tau_{\mathrm{mm}}=2/3) of ALMA band 5, 6 and 7 and disk photosphere (see \se{t_relax}) are plotted along with the temperature contour.$
$    In the middle row, the region where no condensation occurs is plotted in white. In the bottom row, two lines denoting ice(vapor)-to-gas ratio of 10^{-3} are indicated to highlight the snowline region. }$
$    \label{fig:2D_structure}$
$\end{figure*}$
$\label{sec:2D_structure}$
$To illustrate the evolution of disk's thermal structure during the outburst dimming phase, we present 2D snapshots of our best-fit simulation results at different times since L_{\mathrm{bol}} declines in \fg{2D_structure}. We focus on three epochs: t^{\prime} = 110~\mathrm{yr} (the observational epoch) and two additional snapshots taken 50 yr before (when recondensation just begins) and after (when most vapor has recondensed in the outer disk) this point (see \fg{water_sigma}). At each time, the disk temperature, vapor condensation timescale (t_{\mathrm{cond}}) and number density of water vapor (n_{\mathrm{H}_{2}\mathrm{O}}) are shown. The condensation timescale is newcommandined as t_{\mathrm{cond}}=\rho_{\mathrm{vap}}/\mathcal{R}_{\mathrm{cond}}.$
$As L_{\mathrm{bol}} decreases, the temperature drops in the entire disk, with the midplane cooling slower due to the large infrared optical depth. This difference in cooling time renders the midplane hotter than the upper layer, which is also found in \citet{LaznevoiEtal2025}.$
$Concurrently, vapor recondenses and the snowline retreats, as shown in the bottom row of \fg{2D_structure}.$
$The middle row of \fg{2D_structure} shows the condensation timescale. In the upper layer of the disk, t_{\mathrm{cond}} is significantly longer than near the midplane, leaving uncondensed vapor plumes in the disk atmosphere (bottom row). This explains the slowly declining tail in the time evolution of the water column density (\fg{water_sigma}). This "inertia" in vapor's response to changing luminosity was recently observed in DQ Tau, where the cold water abundance shows little reaction to variations in accretion luminosity on timescales of days to weeks \citep{KospalEtal2025}.$
$Radially, t_{\mathrm{cond}} reaches a minimum value at {\sim}80{-}100 au, increasing outward due to lower pebble densities and inward due to higher temperature. Notably, this increase in condensation timescale was also reported by \citet{SmithEtal2025} in their modeling of EX Lup, implied by the higher water abundance towards the outer disk (their Fig.10).$
$The ice freezing-out timescale in EX Lup, constrained by multi-epoch observations of \textit{Spitzer} and JWST \citep{SmithEtal2025}, is one magnitude shorter (\sim 10 yr) than calculated from our simulations. This difference is expected, given the much larger pebble number density in EX Lup's inner disk.$
$Finally, we plot the emission surfaces in our 2sRT calculation (upper row, middle panel), where the millimeter optical depth (\tau_{\mathrm{mm}}) equals 2/3. There is a clear elevation of the emission surface across snowline at \approx 60 au, rising from the recondensation of vapor. As the emission surfaces trace different disk layers thus different temperature, we expect that the continuum emission will not only depend on the surface density distribution but also be modulated by the disk temperature structure. \md{For molecular emission of water at band 5, because \tau_{\mathrm{mm}} \approx 0.7 at the midplane of 80 au, we only expect minor dust attenuation on measuring the mean water column density at 80-120 au.}$
$\section{MCMC analysis}$
$\label{sec:MCMC_result}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{fig/corner_dynamic_revise_45_85.pdf}$
$    \caption{Corner plot of retreating snowline model. The dashed lines indicate 16th, 50th and 84th percentiles of the posterior distribution.}$
$    \label{fig:corner_outburst}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.8\textwidth]{fig/corner_static_revise_45_85.pdf}$
$    \caption{Corner plot of static snowline model.}$
$    \label{fig:corner_static}$
$\end{figure*}$
$To fit the ALMA observations, we adopt a \md{log-normal} likelihood for each dataset:$
$\begin{equation}$
$\label{eq:likelihood_i}$
$    \ln \mathcal{P}_{n}^{i}(x) =  -\frac{1}{2} \left(\frac{\ln x_{n}^{i} - \ln x_{n,\mathrm{syn}}^{i}}{\sigma_{\ln,n}^{i}} \right)^2 - \frac{1}{2} \ln(2 \pi{\sigma^{i}_{\ln,n}}^{2}),$
$\end{equation}$
$where x_{n}^{i} is either the azimuthally averaged continuum intensity of the n-th \md{annulus} in band i (I) or the mean water column density between 80-120 au (N_{\mathrm{H}_{2}^{18}\mathrm{O}}). \md{Correspondingly, x_{n,\mathrm{syn}}^{i} is the synthetic data from the simulations.} \md{Following \citet{ViscardiEtal2025},  \sigma_{\ln,n}^{i} = \sqrt{B_{i}/A_{n}} \times \sigma_{x,n}^{i} / x_{n}^{i} refers to the fractional rms error of the mean within annulus n, where B_{i} is the beam area of band i and A_{n} is the area corresponding to annulus n. For the {\mathrm{H}_{2}^{18}\mathrm{O}} emission, the area spanning 80 to 120 au is treated as a single annulus.}$
$Then the joint log-likelihood reads:$
$\begin{equation}$
$    \label{eq:likelihood}$
$    \ln \mathcal{P} = \sum_{i=\mathrm{B6,B7},n} \ln \mathcal{P}_{n}^{i} (I) + \ln \mathcal{P}_{1}^{\mathrm{B5}} (N_{\mathrm{H}_{2}^{18}\mathrm{O}}),$
$\end{equation}$
$With this joint likelihood, MCMC analysis is performed for both the retreating snowline model and the static snowline model. In the static model, the parameters t^{\prime} and f_{\mathrm{cond}} are replaced by a single L_{\mathrm{bol}}, which controls the temperature structure together with the opacity \kappa_{\mathrm{R}} in the IR band (\se{T_model}). Gaussian priors are used for t^{\prime} (\sigma_{t^{\prime}} = 10 yr) and L_{\mathrm{bol}} (\sigma_{L_{\mathrm{bol}}}=110~L_{\odot}), assuming that the uncertainties in the luminosity curve span 2\sigma range (i.e., the luminosity curve spans \md{40} yr). Other parameters adopt uniform priors. For Z_{\mathrm{peb}}, \mathrm{St}_{0}, f_{\mathrm{cond}} and \kappa_{\mathrm{R}}, sampling is carried out in logarithmic space.$
$We run MCMC until all parameters converge over {>}50 correlation timescales.$
$The resulting posterior distributions are shown in \fgs{corner_outburst}{corner_static}, with values summarized in \tb{mcmc_paras}.$
$\md{In the retreating snowline model, several correlations can be seen among disk parameters (Z_{\mathrm{peb}}, \gamma, \gamma_{\mathrm{peb}} and \mathrm{St}_{0}) and the gas density slope \gamma remains poorly constrained. These degeneracies are expected since the adopted disk model parameters do not directly correspond to the observational diagnostics. Specifically, the following features are noted:$
$\begin{enumerate}$
$    \item The parameter St_{0}, which determines the maximum grain size, is positively correlated with Z_{\mathrm{peb}} and negatively correlated with \gamma and \gamma_{\mathrm{peb}}. First, the Stokes number of pebbles decreases in denser gas. A shallower (larger) \gamma, which implies a higher gas density at 60 au, thus requires a smaller St_{0} to maintain the same s_{\max} of \sim cm, as suggested by the spectral indices (see \se{intensity_shoulder}). Second, pebble size affects the continuum emission differently across the disk. In the inner disk where s_{\max}\gg \mathrm{mm}, increasing the pebble size significantly reduces the mm opacity. In the outer disk where s_{\max} \approx \mathrm{mm}, however, opacity becomes less sensitive to s_{\max} (e.g., \citealt{BirnstielEtal2018}). Consequently, increasing St_{0} both reduces the overall continuum intensity and flattens its radial slope, necessitating larger Z_{\mathrm{peb}} and a steeper surface density profile (lower \gamma_{\mathrm{peb}}) to fit the data.$
$    \item The gas density slope \gamma is negatively correlated with Rosseland mean opacity \kappa_{\mathrm{R}}. This arises since to reproduce the intensity shoulder, the snowline should retreat to \approx60 au at the expected observation time (t^{\prime}), following the assumed luminosity curve. Because the gas density and \kappa_{\mathrm{R}} together govern the disk's cooling time (t_{\mathrm{relax}}), that is, the retreat speed, \gamma and \kappa_{\mathrm{R}} are naturally related. Furthermore, the posterior of \kappa_{\mathrm{R}} essentially follows the assumed Gaussian prior of t^{\prime}.$
$    \item Large \gamma (toward the upper bound) is preferred. This can be understood as follows: during the outburst dimming phase, the outer disk cools faster than the inner regions, steepening the temperature gradient over time \citep{LaznevoiEtal2025}. A higher \gamma (shallower density profile) therefore tends to equalize the cooling rate across the disk, "tilting" the intensity profile towards a plateau, as being observed. A value \gamma>-1 may fit the data even better, but such flat profiles are implausible.$
$    \item The factor f_{\mathrm{cond}}, which controls the condensation rate, is negatively correlated with \kappa_{\mathrm{R}}. An increase in \kappa_{\mathrm{R}} slows the retreat of the snowline (larger t^{\prime}), allowing a reduced f_{\mathrm{cond}} to achieve similar condensation levels.$
$\end{enumerate}$
$From \fg{obv_compare}, we find that the model underestimates the emission within 40 au and correspondingly, struggles to capture the declining spectral index towards inner disk. These discrepancies likely arise from intense viscous heating \citep{AlarconEtal2024} and optically thick environment \citep{TobinEtal2023} in V883 Ori's inner disk., which is included in the model.$
$Simple power-law profiles, as adopted in this work, cannot simultaneously reproduce both the inner and outer disk, implying a sharp transition in the disk properties of V883 Ori.$
$}$
$In the static snowline model, many parameters are driven toward the edges of their prior ranges, indicating that the model cannot adequately reproduce the data. In particular, the disk struggles to reach sufficiently high temperatures. Consequently, \kappa_{\mathrm{R}} \md{and \gamma} are pushed to \md{their} minimum value, and L_{\mathrm{bol}} is constrained to the upper wing of its Gaussian prior, trying to obtain maximum penetration of stellar photons into the disk midplane regions.$
$\end{document}}}$
$\newcommand{\mdd}{\textbf{}}$
$\newcommandinecolor{mygray}{gray}{0.6}$
$\newcommand{\ywc}[1]{[\textcolor[RGB]{128,0,128}{\small YW: \textit{#1}}]}$
$\newcommand{\ywadd}[1]{\textcolor[RGB]{128,0,128}{#1}}$
$\newcommand{\ywch}[2]{\textcolor{mygray}{\sout{#1}}{\textcolor[RGB]{139,0,139}{{#2}}}}$
$\newcommand{\ywrem}[1]{\textcolor{mygray}{\sout{#1}}}$
$\newcommand{\App}[1]{Appendix~\ref{sec:#1}}$
$\newcommand{\app}[1]{App.~\ref{sec:#1}}$
$\newcommand{\Se}[1]{Section~\ref{sec:#1}}$
$\newcommand{\se}[1]{Sect.~\ref{sec:#1}}$
$\newcommand{\Fg}[1]{Figure~\ref{fig:#1}}$
$\newcommand{\fg}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{\fgs}[2]{Figs.~\ref{fig:#1} and \ref{fig:#2}}$
$\newcommand{\tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\Tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\eq}[1]{equation~(\ref{eq:#1})}$
$\newcommand{\Eq}[1]{Equation~(\ref{eq:#1})}$
$\newcommand{\Eqs}[2]{Equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqs}[2]{equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqto}[2]{equations~(\ref{eq:#1})--(\ref{eq:#2})}$
$\newcommand\gas{\mathrm{g}}$
$\newcommand\dust{\mathrm{d}}$
$\newcommand\parti{\mathrm{p}}$
$\newcommand\diff{\mathrm{d}}$
$\iftrue$
$\newcommand{\ywc}[1] $
$\newcommand{\ywrem}[1]{\xspace}$
$\newcommand{\ywch}[2]{#2}$
$\newcommand{\ywadd}[1]{#1\xspace}$
$\newcommand{\cim}[1] $
$\newcommand{\?} $
$\newcommand{\corem}[1]{\xspace}$
$\newcommand{çc}[1]{\xspace}$
$\newcommand{\coch}[2]{#2}$
$\newcommand{\coadd}[1]{#1\xspace}$
$\newcommand{\md} $
$\newcommand{\mdd} $
$\fi$
$\begin{document}$
$   \title{Dust and Water in V883 Ori: Relics of a Retreating Snowline}$
$   \author{$
$            Y.~Wang (\begin{CJK*}{UTF8}{gbsn}王雨\end{CJK*})$
$          \inst{1} \orcidlink{0009-0004-2217-4439}$
$          \and$
$          C.W.~Ormel\inst{1} \orcidlink{0000-0003-4672-8411}$
$          \and$
$          H.-C. Jiang (\begin{CJK*}{UTF8}{gbsn}蒋昊昌\end{CJK*})\inst{2} \orcidlink{0000-0003-2948-5614}$
$          \and$
$          S.~Krijt\inst{3} \orcidlink{0000-0002-3291-6887}$
$          \and$
$          A.~Houge\inst{4} \orcidlink{0000-0001-8790-9011}$
$          \and$
$          E.~Macías\inst{5} \orcidlink{0000-0003-1283-6262}$
$          }$
$        \institute{Department of Astronomy, Tsinghua University, 30 Shuangqing Rd, Haidian DS, 100084 Beijing, China\ 	\email{wang-y21@mails.tsinghua.edu.cn}$
$        \and$
$            Max Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany$
$        \and$
$            Department of Physics and Astronomy, University of Exeter, Exeter, EX4 4QL, UK$
$         \and$
$            Center for Star and Planet Formation, GLOBE Institute, University of Copenhagen, Øster Voldgade 5-7, DK-1350 Copenhagen, Denmark$
$        \and$
$            European Southern Observatory, Karl-Schwarzschild-Str 2, 85748 Garching, Germany}$
$   \date{Received September 15, 1996; accepted March 16, 1997}$
$\abstract{$
$    V883 Ori is an FU-Orionis-type outburst system characterized by a shoulder at 50-70 au in its ALMA band 6 and 7 intensity profiles. Previously, this feature was attributed to dust pile-up from pebble disintegration at the water snowline. However, recent multi-wavelength observations show continuity in the spectral index across the expected snowline region, disfavoring abrupt changes in grain properties. Moreover, extended water emission is detected beyond 80 au, pointing to a snowline further out. This Letter aims to explain both features with a model in which the snowline is receding.$
$    We construct a 2D disk model that solves the cooling and subsequent vapor recondensation during the post-outburst dimming phase.$
$    Our results show that both the intensity shoulder and the extended water emission are natural relics of a retreating snowline: the shoulder arises from excess surface density generated by vapor recondensation at the moving condensation front, while the outer water vapor reservoir persists due to the long recondensation timescales of 10^{2}-10^{3} yr at the disk atmosphere.$
$    As V883 Ori continues to fade, we predict that the intensity shoulder will migrate inward by an observationally significant amount of 10 au over about \md{25} years.}$
$   \keywords{Protoplanetary disks --$
$                Planets and satellites: formation --$
$                Stars: protostars$
$               }$
$   \maketitle$
$\section{Introduction}$
$In protoplanetary disks, snowlines mark the locations where volatile ice sublimates. This process dictates volatile distributions and alters grain properties in the disk (e.g., \citealt{OebergEtal2023}).$
$In this context, outburst systems are ideal laboratories for studying the effect of snowlines. Due to an abrupt increase in accretion, these young stellar objects brighten by many orders of magnitude, heating up the disk for tens to hundreds of years and enabling large-scale sublimation of ices (e.g., \citealt{FischerEtal2023}). In particular, the snowline is pushed to \sim10--100 au, bringing it within reach of facilities such as the Atacama Large Millimeter/submillimeter Array (ALMA).$
$V883 Ori is one of the best-studied FU Orionis outburst systems. \citet{CiezaEtal2016} identified a "dark annulus" in ALMA band 6 continuum,  \md{suggesting} that its water snowline lies at \approx40 au. This shoulder-like feature in the intensity profile was initially attributed to the accumulation of small grains inside the snowline, produced by the \md{disintegration} of dust grains after ice sublimation \citep{SaitoSirono2011, AumatellWurm2011}.$
$However, multi-wavelength analysis has revealed \md{continuity in the} spectral index across the intensity shoulder, implying no abrupt change of the grain physical properties by ice sublimation \citep{HougeKrijt2023,HougeEtal2024}.$
$The exact snowline location is further complicated by the detection of H_{2}^{18}O \citep{TobinEtal2023} and indirect tracers like methanol or HCO^{+} \citep{van'tHoffEtal2018, LeemkerEtal2021}, which suggest a snowline location {\gtrsim}80 \mathrm{au}.$
$In this Letter, we demonstrate that these observations can be consistently explained by a model in which the snowline retreats \md{along} with the decline of bolometric luminosity. At the \md{inward-}moving snowline front, the intensity shoulder \md{arises from} the \md{increase in} surface density of solids due to ice recondensation, while in the outer disk the sluggish recondensation of vapor preserves a relic of a previously hot state.$
$\section{Model}$
$\label{sec:model}$
$We set up the disk at the outburst active phase, when the bolometric luminosity reaches its peak value (see \se{dim_curve}), assuming the dust-vapor mixture is in hydrostatic balance and thermal equilibrium. The main focus of this work is to model the subsequent dimming phase, when L_{\mathrm{bol}} declines and vapor recondenses onto grain surfaces. A key assumption is that dimming proceeds rapidly ({\sim}200~\mathrm{yr}; see \se{dim_curve}) so that dynamic processes (gas advection, pebble drift, and diffusion) can be neglected.$
$\subsection{Disk structure}$
$\label{sec:disk_model}$
$The gas surface density profile is taken as a power law$
$\begin{equation}$
$\label{eq:sigma_gas}$
$    \Sigma_{\gas} (r) = \Sigma_{c} \left( \frac{r}{r_{c}}\right)^{-\gamma}.$
$\end{equation}$
$Following the fit of dust thermal emission by \citet{CiezaEtal2018}, \md{we take r_{c} = 31 \mathrm{au} and \Sigma_{c}=160~\mathrm{g~cm^{-2}}, while \gamma is a treated as a parameter given its poor constraint \citep{van'tHoffEtal2018}}. The surface density of pebbles is taken as:$
$\begin{equation}$
$\label{eq:sigma_peb}$
$    \Sigma_{\parti}= Z_{\mathrm{peb}} \Sigma_{\gas}(r_{c}) \left( \frac{r}{r_{c}}\right)^{-\gamma_{\mathrm{peb}}},$
$\end{equation}$
$where Z_\mathrm{peb} is the metallicity at the reference position.$
$Beyond the snowline, pebbles are \md{assumed} 50\% \md{refractory} and 50\% water ice by mass, \mdd{with the material densities of constituent species following DSHARP \citep{BirnstielEtal2018}}.$
$Following \citet{WangEtal2025} we construct a 2D model in the disk's R{-}z plane, where a standard viscous disk with \alpha=10^{-3} \citep{ShakuraSunyaev1973} is assumed. For solids, we adopt an MRN size distribution \citep{MathisEtal1977}, which is appropriate when dust growth is limited by fragmentation \citep{BirnstielEtal2011}. The size distribution is characterized by an upper bound s_{\max}, corresponding to a midplane Stokes number \mathrm{St}_{0}, a lower bound of s_{\min}=0.1 \mu\mathrm{m}, and is assumed unchanged by the outburst \citep{HougeEtal2024}.$
$Pebbles of different sizes are vertically distributed according to their scale height H_{\mathrm{p},i}(H_\mathrm{g}, \mathrm{St}_{i,0},\alpha), where H_{\gas} is the gas pressure scale height and \mathrm{St}_{i,0} the Stokes number of pebble i at the midplane (see \app{dust_bins}).$
$The vapor condensation rate (\mathcal{R}_{\mathrm{cond}}) is determined by its saturation condition and the total surface area of particles (e.g., \citealt{RosJohansen2013, SchoonenbergOrmel2017}). Specifically, the vapor density at any point in the disk reduces at a rate,$
$\begin{equation}$
$\label{eq:limiter}$
$    -\frac{\diff \rho_\mathrm{vap}}{\diff t}$
$    = \mathcal{R}_{\rm cond}$
$    = \langle \pi s^{2}_{\parti} n_\parti \rangle v_{\mathrm{th,vap}} \rho_{\mathrm{vap}} \left(1 - \frac{P_{\mathrm{eq}}}{P_{\mathrm{vap}}}\right).$
$\end{equation}$
$where \langle \pi s^{2}_{\parti} n_\parti \rangle is the surface area of pebbles per unit volume (\app{dust_bins}, \eq{surface_area}).$
$\subsection{Luminosity during outburst dimming phase}$
$\label{sec:dim_curve}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/dimming_curve_revise.pdf}$
$    \caption{Evolution of the bolometric luminosity of V883 Ori during the outburst dimming phase. The symbols denote measurements from different studies, \md{with horizontal bars showing the time range of adopted data}. The blue band represents the adopted luminosity curve, which carries an uncertainty of {\sim}40~\mathrm{yrs} due to the uncertainties in estimating L_{\mathrm{bol}}.}$
$    \label{fig:dimming_curve}$
$\end{figure}$
$We model the dimming of V883 Ori following the published luminosities, accounting for observational uncertainties arising from dust extinction and instrumental saturation (see \app{L_bol_uncertain}). Specifically, the time-dependent bolometric luminosity reads,$
$\begin{equation}$
$    \label{eq:Lfunc}$
$    L_{\mathrm{bol}} (t) = L_{\star} + (L_{\mathrm{pk}} - L_{\star}) \exp\left[-0.5\left(\frac{t-t_{\mathrm{beg}}}{50~\mathrm{yr}}\right)^{2} \right],$
$\end{equation}$
$where L_{\star} is the stellar luminosity (6 L_{\odot}; \citealt{CiezaEtal2016}). We adopt a peak luminosity of L_{\mathrm{pk}}=4 000 L_{\odot} (comparable to that used in \citealt{LeemkerEtal2021}), which pushes the water snowline to 80--100~au. The parameter t_{\mathrm{beg}} indicates the year when the outburst begins to fade.  As shown in \fg{dimming_curve}, choosing t_{\mathrm{beg}}=\md{1880} A.D. makes the dimming curve consistent with the lower limit of the estimated L_{\mathrm{bol}} of V883 Ori (400 L_{\odot}, \citealt{StromStrom1993}; and 218 L_{\odot}, \citealt{FurlanEtal2016}). By shifting t_{\mathrm{beg}} forward by 40 \mathrm{yr} (A.D.~1920), the curve reaches the upper limit (647 L_{\odot}) given by \citet{LiuEtal2022}. Recently, \citet{CarvalhoEtal2025} fit the spectrum at 0.4-4.2 \mu \mathrm{m} and obtained an accretion luminosity of 458 L_{\odot}, lying well within the assumed range. Through \eq{Lfunc}, the model output is only a function of t'=t-t_\mathrm{beg}, the time since L_{\mathrm{bol}} starts to decline.$
$\subsection{Temperature model}$
$\label{sec:T_model}$
$We adopt a two-stream radiation transfer (2sRT) method to calculate the temperature (e.g., \citealt{Hubeny1990}).$
$The heating sources at each grid cell are due to irradiation (q_{\mathrm{irr}}), viscous dissipation (q_\mathrm{vis}) and latent heat exchange (q_{\mathrm{latent}}).$
$We compute q_{\mathrm{irr}} from the bolometric luminosity (\eq{Lfunc}), while the viscous heating rate follows from the \alpha-disk viscosity \md{(q_{\mathrm{vis}}=\frac{9}{4} \rho \alpha c_{\mathrm{s}}^{2} \Omega)} and the latent heating rate is proportional to the \md{condensation} rate (\eq{limiter}, see \citealt{WangEtal2025}).$
$In the outburst active phase, the disk adjusts to a temperature and density structure determined by the bolometric luminosity L_\mathrm{pk}, assuming hydrostatic balance.$
$In the dimming phase, following \citet{FlockEtal2017}, we let the disk adapt to a new equilibrium temperature obtained by the 2sRT (T_{\mathrm{eq}}) over a thermal relexation timescale, \mathrm{d} T /\mathrm{d}t = (T_{\mathrm{eq}} - T) /t_{\mathrm{relax}} (see \app{t_relax} for the calculation of t_{\mathrm{relax}}). The Rosseland mean opacity per unit gas mass \kappa_{\mathrm{R}} -- a key quantity that determines t_{\mathrm{relax}} -- is a free model parameter. In our simulation (see \se{results}), due to the low metallicity and disk viscosity, the disk temperature is \md{determined} by irradiation, with q_\mathrm{vis} and q_{\mathrm{latent}} contributing little.$
$\subsection{Fitting ALMA observations}$
$\label{sec:fit_alma}$
$The intensity shoulder in V883 Ori is identified in both ALMA band 6 (0.038", project code: 2015.1.00350.S, PI: Lucas Cieza) and band 7 (0.028", project code: 2016.1.00728.S, PI: Lucas Cieza) high-resolution continuum images.$
$\md{We converted the continuum images into radial profiles by azimuthally averaging the data within concentric annuli of radial width 1/5 of the respective beam size. To fit the continuum profiles,} we generate synthetic intensity profiles in both bands by conducting 2sRT with \md{profiles for the} dust temperature and density taken from the simulation results. The 2sRT is performed in plane-parallel manner along the disk's vertical direction, with the optical depth corrected for disk inclination i=38.3^{\circ} \citep{CiezaEtal2016}. Pebble opacities for different sizes and ice fractions are computed with \texttt{optool} \citep{DominikEtal2021}, adopting the DSHARP optical constants \citep{BirnstielEtal2018}. \md{After being convolved with the beam size, the synthetic intensity profiles are compared with the continuum data at 45–85 au, where the intensity shoulder lies}.$
$\citet{TobinEtal2023} measured the column density of water isotope H_{2}^{18}O in band 5 (0.126"). Given the limited data sensitivity, we fit only the mean column density between 80–120 au in our simulations to the observed value, assuming an isotopic ratio ^{18}O/^{16}O = 1/560 \citep{WilsonRood1994}. In reality, the observed water vapor abundance may be influenced by processes beyond simple recondensation (\se{water_emission}). Therefore, we introduce an additional free parameter f_{\mathrm{cond}} such that, - \dust \rho_{\mathrm{vap}} / \dust t = f_{\mathrm{cond}} \mathcal{R}_{\mathrm{cond}}.$
$To incorporate observational constraints from both continuum and water emission, we apply a \md{log-normal} likelihood for each dataset, \md{and} weight each likelihood with \md{the number of beams covered by the area of interest} (see \eqs{likelihood_i}{likelihood}).$
$\section{Results}$
$\label{sec:results}$
$\newcommand{\arraystretch}{1.2}$
$\begin{table}[]$
$    \centering$
$    \caption{Overview of MCMC parameters and posterior values \md{([16,50,84]-th percentiles)} for both the retreating snowline and static snowline model.}$
$    \begin{tabular}{c|l|c|c}$
$        \hline$
$         & Prior & \multicolumn{2}{c}{\makecell{Posterior \ retreating / static}} \         \hline$
$       \log Z_{\mathrm{peb}} & [-3.5, -2] & -2.89^{+0.05}_{-0.05} & -2.84^{+0.007}_{-0.007}  \         \gamma & [-2.0, -1.0] & -1.31^{+0.22}_{-0.34}  & -1.99^{+0.005}_{-0.002} \         \gamma_{\mathrm{peb}} & [-1.5, -0.5] & -0.97^{+0.10}_{-0.10} & -0.50^{+0.004}_{-0.008} \         \log St_{0} & [-3, -1] & -1.81^{+0.13}_{-0.12} & -2.71^{+0.016}_{-0.010} \         t^{\prime} (yr) & [60, 170] & 120.8^{+9.3}_{-9.0} & -  \         \log f_{\mathrm{cond}} & [-0.5, 1.5] &  0.53^{+0.09}_{-0.08} & -\         \log \kappa_{\mathrm{R}} & [0, 1.3] & 0.67^{+0.17}_{-0.15} & 0.006^{+0.001}_{-0.000} \         \hline$
$        L_{\mathrm{bol}} (L_{\odot}) & [0, 978] & - & 489.9^{+6.4}_{-6.0} \         \hline$
$    \end{tabular}$
$    \tablefoot{Z_{\mathrm{peb}}: pebble metallicity (Eq.\ref{eq:sigma_peb}); \gamma, \gamma_{\mathrm{peb}}: gas and pebble surface density index (Eq.\ref{eq:sigma_gas},\ref{eq:sigma_peb}); St_{0}: Stokes number of maximum grain size (\se{disk_model}, \ref{sec:dust_bins}); t^{\prime}: time since the outburst dimming (Eq.\ref{eq:Lfunc});$
$    f_{\mathrm{cond}}: condensation rate correction factor (\se{fit_alma}); \kappa_{\mathrm{R}}: Rosseland-mean opacity (\se{T_model}, \ref{sec:t_relax}); L_{\mathrm{bol}}: bolometric luminosity, for static model (\se{static_model}).}$
$    \label{tab:mcmc_paras}$
$\end{table}$
$We conduct Markov chain Monte Carlo (MCMC) analysis with \textit{emcee} \citep{Foreman-MackeyEtal2013} to obtain the posterior distribution (see \se{MCMC_result}). The \md{parameter's} posterior values are given in \tb{mcmc_paras}.$
$\subsection{Emergence of intensity shoulder}$
$\label{sec:intensity_shoulder}$
$The best-fit results are shown in \fg{obv_compare}, where the synthetic intensity profiles agree well with the dual-band data, especially capturing the intensity shoulder feature at 50-70 au (gray-shaded region). To understand this, we plot the time evolution of the corresponding surface density profiles in \fg{sigma_time}. As the disk cools during the outburst dimming phase, vapor gradually recondenses on pebbles, creating a bump in surface density at the condensation front, which appears in continuum as the intensity shoulder. Matching the observed position of the shoulder requires the snowline to retreat to \approx60 au at the observational epoch (t'=121 \mathrm{yr}). This places a stringent constraint on the thermal relaxation time, or IR-opacity \kappa_{\mathrm{R}}, of the disk. If the disk had cooled more rapidly (lower \kappa_R), the snowline would already have retreated inside 50 au, shifting the shoulder much closer to the star, within the expected observational time.$
$Simultaneously fitting the dual-band continuum yields \mathrm{St_{0}} \approx 0.01, corresponding to a maximum grain size of {\sim}centimeter. The need of having cm-sized particles in the disk follows from the low spectral indices at 50-70 au (\alpha=2.5-3.0), a result consistently found across ALMA band 3-7 in V883 Ori by \citet{HougeEtal2024}.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/sigma_time_revise_45_85.pdf}$
$    \caption{Time evolution of the surface density profiles of the best-fit model. Silicate surface density remains constant with time. The thick line denotes the best-fit t^{\prime}=121 yr.}$
$    \label{fig:sigma_time}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/bestfit_intens_revise_45_85.pdf}$
$    \caption{Comparison of the synthetic intensity profiles with ALMA continuum. Crosses show ALMA data adopted from \citet{HougeEtal2024}, spaced according to the beam sizes; shaded regions indicate the rms error. Solid lines represent the best-fit retreating snowline model, while dotted lines correspond to the best-fit static model. Enhanced emission towards the inner regions arises likely from intense viscous heating \citep{AlarconEtal2024}, which is not accounted for in the model.}$
$    \label{fig:obv_compare}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\columnwidth]{fig/H2O18_time_revise_45_85.pdf}$
$    \caption{Average column density of H_{2}^{18}O at 80-120 au under different condensation rate (f_{\mathrm{cond}}). The vertical grey band indicates the time range when observing the intensity shoulder. The dashed \md{black} line denotes the average column density derived from ALMA band 5 data by \citet{TobinEtal2023} with 1\sigma uncertainty shaded in grey. \md{Dotted grey line denotes the best-fit value in static snowline model.}}$
$    \label{fig:water_sigma}$
$\end{figure}$
$\subsection{Extended water emission}$
$\label{sec:water_emission}$
$The mean column density of H_{2}^{18}O at 80-120 au, as estimated from our models assuming optically thin emission, is presented in \fg{water_sigma}.$
$In all simulated cases, the water abundance drops sharply within the first 110 years, and transitions to a slowly-evolving stage afterwards. This is due to different condensation timescales in the disk's midplane and the upper layers (see \app{2D_structure}). After the snowline retreats, vapor in the midplane recondenses rapidly. Conversely, in the disk's upper layer, where pebble densities are much lower, the recondensation time can reach {\sim}10^2{-}10^3 \mathrm{yr} \citep{VisserEtal2015,RabEtal2017}.$
$Therefore, due to the lag caused by the finite recondensation time, the snowline inferred from molecular emission would consistently appear further away than its actual position during the outburst dimming phase.$
$Our modeling shows that an enhanced condensation rate (f_\mathrm{cond}\approx4) is required to match the inferred water vapor abundance, which may indicate a shallower size distribution and/or porous grains \citep{Birnstiel2024}, both of which increase the total dust surface area at the disk atmosphere.$
$In addition, the destruction of water by photodissociation, heterogeneous nucleation and different outburst duration may also influence the observed gas-phase water abundance \citep{FischerEtal2023,RosJohansen2024,Romero-MirzaEtal2024i}.$
$\subsection{The need for a retreating snowline model}$
$\label{sec:static_model}$
$A retreating snowline naturally explains the distant intensity shoulder and the extended water emission, relics of the earlier outburst.$
$In contrast, a static snowline model fails to explain the observations (\fg{obv_compare} and \fg{corner_static}).$
$To illustrate this, we performed another MCMC simulation, where the disk structure is set by a constant L_{\mathrm{bol}} (treated as a free parameter) rather than evolving with the luminosity curve. In other words, the temperature and vapor equilibrium instantly adapt to the stellar luminosity.$
$\md{Because of the instantaneous adjustment, the static model fails to maintain a sufficient amount of vapor in the outer disk: the best-fit water number density is \approx7 \sigma away from the observed value (\fg{water_sigma}). Furthermore, in the static model, the snowline is located at \approx30 au, failing to reproduce the shoulder feature at 50-70 au in the observed intensity profile.}$
$\section{Conclusions and discussions}$
$\label{sec:conclusion}$
$We investigated V883 Ori's continuum and water line emission by a retreating snowline model.$
$The key findings are:$
$   \begin{enumerate}$
$       \item The dimming stage of a stellar outburst results in a receding water snowline. Vapor condensation at the snowline front creates a bump in surface density, marking the transition from icy to ice-free pebbles.$
$       \item Vapor in the disk upper layer takes {\sim}10^{2}{-}10^3 \mathrm{yr} to recondense, leading to substantial amounts of water vapor in the outer disk, even after the snowline has retreated inwards. The observed H_2^{18}O outside the current snowline region (50-70 au) is a leftover of a much hotter past phase.$
$       \item The retreating snowline model simultaneously reproduces the intensity shoulder feature in ALMA band 6/7 continuum and the extended water emission. In contrast, a static model fails to maintain the snowline at 50-70 au under the present-day luminosity.$
$\end{enumerate}$
$In our modeling, both the disk cooling time (t_{\mathrm{relax}}), which controls the snowline retreat speed, and the mm opacity, which sets the continuum emission, depends on dust opacities. Thus, the thermal history of outburst systems constrains dust properties. For example, the retreating snowline model requires \kappa_{\mathrm{R}} \approx 4.6~\mathrm{cm^{2}~g^{-1}}, whereas a self-consistent calculation from the DSHARP opacity yields \kappa_{\mathrm{R}} \approx 0.4~\mathrm{cm^{2}~g^{-1}}.$
$This mismatch implies excess IR opacity relative to millimeter, possibly arising from a non-MRN size distribution, carbon-rich small grains \citep{WoitkeEtal2016}, or porous dust \citep{ZhangEtal2023}.$
$\md{Under the assumed dimming curve of luminosity}, our model predicts that the intensity shoulder will move by 10 au over \md{25} years (\fg{sigma_time}), a trend testable with future observations. With ALMA’s growing sample of outburst systems at high resolution, we expect the intensity shoulder to emerge as an ubiquitous feature of large-scale water phase transitions in disks.$
$\begin{acknowledgements}$
$    The authors thank for useful discussion with Vardan Elbakyan, Shangjia Zhang, Hanpu Liu. This work is supported by the National Natural Science Foundation of China under grant Nos. 12233004 and 12473065.$
$\end{acknowledgements}$
$\bibliographystyle{aa}$
$\bibliography{ads}$
$\appendix$
$\section{Bolometric luminosity measurements}$
$\label{sec:L_bol_uncertain}$
$The onset of the outburst in V883 Ori was not directly observed. But it likely starts before A.D.~1888, when the reflection nebula IC 430 was observed to be illuminated by the outburst \citep{Pickering1890}, as argued by \citet{StromStrom1993}.$
$\md{By fitting the spectral energy distribution, the bolometric luminosity of V883 Ori was estimated to be 400~L_{\odot} \citep{StromStrom1993} in the 1980s} \citep{AllenEtal1975,NakajimaEtal1986} and had significantly decreased to 218~L_{\odot} \citep{FurlanEtal2016, ConnelleyReipurth2018} in about 20 years. Given the typical month-to-year rise time of FUors (e.g., \citealt{FischerEtal2023}), it is very likely that V883 Ori has been in the dimming phase since A.D.~1888.$
$However, estimating the bolometric luminosity of V883 Ori is fraught with substantial uncertainty. As a young stellar object, it suffers severe extinction and reddening at near-infrared wavelengths from dusty proto-stellar envelopes. In the mid-infrared, where extinction is less severe, observations are hindered by heavy saturation in modern surveys like WISE \citep{ContrerasPenaEtal2020}. \citet{LiuEtal2022} derived a much higher bolometric luminosity of {\approx}647 L_{\odot} by accounting for the reddening.$
$\md{Unlike most works adopting discrete photometric data, recently, \citet{CarvalhoEtal2025} fits the medium resolution spectrophotometry within a range of 0.4-2.5 \mu\mathrm{m} and found excellent match to even broader spectrum across 0.4-4.2 \mu\mathrm{m}. They yielded an accretion luminosity of 458 L_{\odot}, which lies well inside the assumed luminosity curve (\fg{dimming_curve}).}$
$\section{Dust model}$
$\label{sec:dust_bins}$
$At each radius, we model the pebble size distribution by constructing several size bins spanning radii from s_{\max} and s_{\min}.$
$For small pebbles (e.g., s_{\parti}{<}0.1~\mathrm{cm}), logarithmic spacing is used, whereas for larger ones, the size bins are more evenly spaced.$
$The pebble surface density for each size bin (\Sigma_{\parti,i}) is then obtained by dividing \Sigma_{\parti} into these size bins according to the MRN distribution \diff N_{\parti} / \diff s_{\parti} \propto s_{\parti}^{-3.5}.$
$For each size bin i, its Stokes number (\mathrm{St}_{i}) is obtained by scaling the pebble size at the bin center (s_{\parti,i}) with s_{\max}, assuming that the pebbles follow the Epstein drag law: i.e., \mathrm{St}_{i,0}= \mathrm{St}_{0} ~s_{\parti,i}/s_{\max}. Then the 2D pebble density distribution is obtained by adopting normal distribution in the vertical direction:$
$\begin{equation}$
$\label{eq:rho_peb}$
$  \rho_{\parti,i} (z) = \rho_{\parti,0} \exp\left[ -\frac{1}{2} \left( \frac{z}{H_{\parti,i}}\right)^2 \right].$
$\end{equation}$
$Here H_{\parti,i} is the scale height of pebbles in the size bin i, following \citet{DubrulleEtal1995}:$
$\begin{equation}$
$\label{eq:H_peb}$
$    H_{\parti,i} (z)= H_{\gas} \left[ 1 + \frac{\mathrm{St}_{i,0}}{\alpha}\left(\frac{2 H_{\gas}^{2}}{z^{2}}\right)\left(\exp \left(\frac{z^{2}}{2 H_{\gas}^{2}}\right)-1\right) \right]^{-1/2}.$
$\end{equation}$
$In \eq{H_peb}, the vertical variation of Stokes number due to gas stratification is accounted for, \mathrm{St}_{i}(z) = \mathrm{St}_{i,0} \rho_{\gas}(z)/\rho_{\gas}(0)  \approx \mathrm{St}_{i,0} \exp \left(z^{2}/ 2 H_{\gas}^{2}\right). The gas scale height H_{\gas} is determined in the outburst active phase, after iterating between the gas density distribution, assuming hydrostatic balance, and the temperature structure calculated from 2sRT.$
$Given \Sigma_{\parti,i}, the midplane pebble density \rho_{\parti,0} can be obtained by integrating the vertical density distribution following \eq{rho_peb}.$
$Recently, \citet{HougeEtal2024} preformed dust evolution modeling and demonstrated that, to match the observed spectral index in V883 Ori, pebble sizes should not evolve appreciably during the swift outburst event. Therefore, we maintain a constant pebble size distribution throughout the outburst dimming phase.$
$Given the size distribution, the total surface area of pebbles per unit volume can be obtained by summing over all size bins,$
$\begin{equation}$
$    \label{eq:surface_area}$
$    \langle \pi s^{2}_{\parti} n_\parti \rangle = \pi\sum_{i} s_{\parti,i}^{2} n_{\parti,i},$
$\end{equation}$
$which is used to compute the vapor recondensation rate \mathcal{R}_{\mathrm{cond}}. The number density of particles in each size bin is given as n_{\parti,i} = \rho_{\parti,i}/m_{\parti,i}, where m_{\parti,i} = 4\pi \rho_{\bullet}  s_{\parti,i}^{3}/3 is the particle mass. We adopt \rho_{\bullet}=1.5~\mathrm{g~cm}^{-3} as the internal density of pebbles, assuming a compact ice-silicate mixture with half ice in mass (e.g., \citealt{SchoonenbergOrmel2017}).$
$\section{Thermal relaxation timescale}$
$\label{sec:t_relax}$
$Following \citet{FlockEtal2017}, we combine the optically thin and optically thick limits to determine the thermal relaxation time:$
$\begin{equation}$
$\label{eq:t_cool}$
$    t_{\mathrm{relax}} = t_{\mathrm{thin}} + t_{\mathrm{thick}} = \frac{l_{\mathrm{mfp}}^{2}}{3D_{\mathrm{rad}}} + \frac{H_{\mathrm{phot}}^{2}}{D_{\mathrm{rad}}}$
$\end{equation}$
$where l_{\mathrm{mfp}} = 1/(\kappa_{\mathrm{R}} \rho_{\gas}) is the mean free path of photons, D_{\mathrm{rad}} = 16 \sigma_{\mathrm{b}} T^{3} / (3\kappa$
$_{\mathrm{R}} \rho_{\gas}^{2} C_{\mathrm{v}}) is the radiation diffusion coefficient and H_{\mathrm{phot}} is the height of disk photosphere (\tau_{\mathrm{R}}=2/3), as measured from the simulation. Here C_{\mathrm{v}} = 8.2\times10^{7}~\mathrm{erg~g^{-1}~K^{-1}} is the specific heat capacity of the He-H_{2} mixture (assuming 71\% H_{2} and ideal gas) and \kappa_{\mathrm{R}} is the Rosseland mean opacity per gas mass, which is a free parameter.$
$\section{2D structure of retreating snowline}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.9\textwidth]{fig/2D_structure_revise_45_85.png}$
$    \caption{Disk temperature, vapor condensation timescale (t_{\mathrm{cond}}) and number density of water (n_{\mathrm{H_{2}\mathrm{O}}}) during the outburst dimming phase of the best-fit model. Here t^{\prime}=t-t_{\mathrm{beg}} is the time lapse since the bolometric luminosity started to decline. The value of  L_{\mathrm{bol}} corresponding to this time is labeled.$
$    In the top row, the range of 140-150 K is shaded in white to highlight the snowline region.$
$    At t^{\prime}=121~\mathrm{yr} -- the present epoch -- the emission surfaces (\tau_{\mathrm{mm}}=2/3) of ALMA band 5, 6 and 7 and disk photosphere (see \se{t_relax}) are plotted along with the temperature contour.$
$    In the middle row, the region where no condensation occurs is plotted in white. In the bottom row, two lines denoting ice(vapor)-to-gas ratio of 10^{-3} are indicated to highlight the snowline region. }$
$    \label{fig:2D_structure}$
$\end{figure*}$
$\label{sec:2D_structure}$
$To illustrate the evolution of disk's thermal structure during the outburst dimming phase, we present 2D snapshots of our best-fit simulation results at different times since L_{\mathrm{bol}} declines in \fg{2D_structure}. We focus on three epochs: t^{\prime} = 110~\mathrm{yr} (the observational epoch) and two additional snapshots taken 50 yr before (when recondensation just begins) and after (when most vapor has recondensed in the outer disk) this point (see \fg{water_sigma}). At each time, the disk temperature, vapor condensation timescale (t_{\mathrm{cond}}) and number density of water vapor (n_{\mathrm{H}_{2}\mathrm{O}}) are shown. The condensation timescale is newcommandined as t_{\mathrm{cond}}=\rho_{\mathrm{vap}}/\mathcal{R}_{\mathrm{cond}}.$
$As L_{\mathrm{bol}} decreases, the temperature drops in the entire disk, with the midplane cooling slower due to the large infrared optical depth. This difference in cooling time renders the midplane hotter than the upper layer, which is also found in \citet{LaznevoiEtal2025}.$
$Concurrently, vapor recondenses and the snowline retreats, as shown in the bottom row of \fg{2D_structure}.$
$The middle row of \fg{2D_structure} shows the condensation timescale. In the upper layer of the disk, t_{\mathrm{cond}} is significantly longer than near the midplane, leaving uncondensed vapor plumes in the disk atmosphere (bottom row). This explains the slowly declining tail in the time evolution of the water column density (\fg{water_sigma}). This "inertia" in vapor's response to changing luminosity was recently observed in DQ Tau, where the cold water abundance shows little reaction to variations in accretion luminosity on timescales of days to weeks \citep{KospalEtal2025}.$
$Radially, t_{\mathrm{cond}} reaches a minimum value at {\sim}80{-}100 au, increasing outward due to lower pebble densities and inward due to higher temperature. Notably, this increase in condensation timescale was also reported by \citet{SmithEtal2025} in their modeling of EX Lup, implied by the higher water abundance towards the outer disk (their Fig.10).$
$The ice freezing-out timescale in EX Lup, constrained by multi-epoch observations of \textit{Spitzer} and JWST \citep{SmithEtal2025}, is one magnitude shorter (\sim 10 yr) than calculated from our simulations. This difference is expected, given the much larger pebble number density in EX Lup's inner disk.$
$Finally, we plot the emission surfaces in our 2sRT calculation (upper row, middle panel), where the millimeter optical depth (\tau_{\mathrm{mm}}) equals 2/3. There is a clear elevation of the emission surface across snowline at \approx 60 au, rising from the recondensation of vapor. As the emission surfaces trace different disk layers thus different temperature, we expect that the continuum emission will not only depend on the surface density distribution but also be modulated by the disk temperature structure. \md{For molecular emission of water at band 5, because \tau_{\mathrm{mm}} \approx 0.7 at the midplane of 80 au, we only expect minor dust attenuation on measuring the mean water column density at 80-120 au.}$
$\section{MCMC analysis}$
$\label{sec:MCMC_result}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{fig/corner_dynamic_revise_45_85.pdf}$
$    \caption{Corner plot of retreating snowline model. The dashed lines indicate 16th, 50th and 84th percentiles of the posterior distribution.}$
$    \label{fig:corner_outburst}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.8\textwidth]{fig/corner_static_revise_45_85.pdf}$
$    \caption{Corner plot of static snowline model.}$
$    \label{fig:corner_static}$
$\end{figure*}$
$To fit the ALMA observations, we adopt a \md{log-normal} likelihood for each dataset:$
$\begin{equation}$
$\label{eq:likelihood_i}$
$    \ln \mathcal{P}_{n}^{i}(x) =  -\frac{1}{2} \left(\frac{\ln x_{n}^{i} - \ln x_{n,\mathrm{syn}}^{i}}{\sigma_{\ln,n}^{i}} \right)^2 - \frac{1}{2} \ln(2 \pi{\sigma^{i}_{\ln,n}}^{2}),$
$\end{equation}$
$where x_{n}^{i} is either the azimuthally averaged continuum intensity of the n-th \md{annulus} in band i (I) or the mean water column density between 80-120 au (N_{\mathrm{H}_{2}^{18}\mathrm{O}}). \md{Correspondingly, x_{n,\mathrm{syn}}^{i} is the synthetic data from the simulations.} \md{Following \citet{ViscardiEtal2025},  \sigma_{\ln,n}^{i} = \sqrt{B_{i}/A_{n}} \times \sigma_{x,n}^{i} / x_{n}^{i} refers to the fractional rms error of the mean within annulus n, where B_{i} is the beam area of band i and A_{n} is the area corresponding to annulus n. For the {\mathrm{H}_{2}^{18}\mathrm{O}} emission, the area spanning 80 to 120 au is treated as a single annulus.}$
$Then the joint log-likelihood reads:$
$\begin{equation}$
$    \label{eq:likelihood}$
$    \ln \mathcal{P} = \sum_{i=\mathrm{B6,B7},n} \ln \mathcal{P}_{n}^{i} (I) + \ln \mathcal{P}_{1}^{\mathrm{B5}} (N_{\mathrm{H}_{2}^{18}\mathrm{O}}),$
$\end{equation}$
$With this joint likelihood, MCMC analysis is performed for both the retreating snowline model and the static snowline model. In the static model, the parameters t^{\prime} and f_{\mathrm{cond}} are replaced by a single L_{\mathrm{bol}}, which controls the temperature structure together with the opacity \kappa_{\mathrm{R}} in the IR band (\se{T_model}). Gaussian priors are used for t^{\prime} (\sigma_{t^{\prime}} = 10 yr) and L_{\mathrm{bol}} (\sigma_{L_{\mathrm{bol}}}=110~L_{\odot}), assuming that the uncertainties in the luminosity curve span 2\sigma range (i.e., the luminosity curve spans \md{40} yr). Other parameters adopt uniform priors. For Z_{\mathrm{peb}}, \mathrm{St}_{0}, f_{\mathrm{cond}} and \kappa_{\mathrm{R}}, sampling is carried out in logarithmic space.$
$We run MCMC until all parameters converge over {>}50 correlation timescales.$
$The resulting posterior distributions are shown in \fgs{corner_outburst}{corner_static}, with values summarized in \tb{mcmc_paras}.$
$\md{In the retreating snowline model, several correlations can be seen among disk parameters (Z_{\mathrm{peb}}, \gamma, \gamma_{\mathrm{peb}} and \mathrm{St}_{0}) and the gas density slope \gamma remains poorly constrained. These degeneracies are expected since the adopted disk model parameters do not directly correspond to the observational diagnostics. Specifically, the following features are noted:$
$\begin{enumerate}$
$    \item The parameter St_{0}, which determines the maximum grain size, is positively correlated with Z_{\mathrm{peb}} and negatively correlated with \gamma and \gamma_{\mathrm{peb}}. First, the Stokes number of pebbles decreases in denser gas. A shallower (larger) \gamma, which implies a higher gas density at 60 au, thus requires a smaller St_{0} to maintain the same s_{\max} of \sim cm, as suggested by the spectral indices (see \se{intensity_shoulder}). Second, pebble size affects the continuum emission differently across the disk. In the inner disk where s_{\max}\gg \mathrm{mm}, increasing the pebble size significantly reduces the mm opacity. In the outer disk where s_{\max} \approx \mathrm{mm}, however, opacity becomes less sensitive to s_{\max} (e.g., \citealt{BirnstielEtal2018}). Consequently, increasing St_{0} both reduces the overall continuum intensity and flattens its radial slope, necessitating larger Z_{\mathrm{peb}} and a steeper surface density profile (lower \gamma_{\mathrm{peb}}) to fit the data.$
$    \item The gas density slope \gamma is negatively correlated with Rosseland mean opacity \kappa_{\mathrm{R}}. This arises since to reproduce the intensity shoulder, the snowline should retreat to \approx60 au at the expected observation time (t^{\prime}), following the assumed luminosity curve. Because the gas density and \kappa_{\mathrm{R}} together govern the disk's cooling time (t_{\mathrm{relax}}), that is, the retreat speed, \gamma and \kappa_{\mathrm{R}} are naturally related. Furthermore, the posterior of \kappa_{\mathrm{R}} essentially follows the assumed Gaussian prior of t^{\prime}.$
$    \item Large \gamma (toward the upper bound) is preferred. This can be understood as follows: during the outburst dimming phase, the outer disk cools faster than the inner regions, steepening the temperature gradient over time \citep{LaznevoiEtal2025}. A higher \gamma (shallower density profile) therefore tends to equalize the cooling rate across the disk, "tilting" the intensity profile towards a plateau, as being observed. A value \gamma>-1 may fit the data even better, but such flat profiles are implausible.$
$    \item The factor f_{\mathrm{cond}}, which controls the condensation rate, is negatively correlated with \kappa_{\mathrm{R}}. An increase in \kappa_{\mathrm{R}} slows the retreat of the snowline (larger t^{\prime}), allowing a reduced f_{\mathrm{cond}} to achieve similar condensation levels.$
$\end{enumerate}$
$From \fg{obv_compare}, we find that the model underestimates the emission within 40 au and correspondingly, struggles to capture the declining spectral index towards inner disk. These discrepancies likely arise from intense viscous heating \citep{AlarconEtal2024} and optically thick environment \citep{TobinEtal2023} in V883 Ori's inner disk., which is included in the model.$
$Simple power-law profiles, as adopted in this work, cannot simultaneously reproduce both the inner and outer disk, implying a sharp transition in the disk properties of V883 Ori.$
$}$
$In the static snowline model, many parameters are driven toward the edges of their prior ranges, indicating that the model cannot adequately reproduce the data. In particular, the disk struggles to reach sufficiently high temperatures. Consequently, \kappa_{\mathrm{R}} \md{and \gamma} are pushed to \md{their} minimum value, and L_{\mathrm{bol}} is constrained to the upper wing of its Gaussian prior, trying to obtain maximum penetration of stellar photons into the disk midplane regions.$
$\end{document}}$
$\newcommand{\ywc}[1]{[\textcolor[RGB]{128,0,128}{\small YW: \textit{#1}}]}$
$\newcommand{\ywadd}[1]{\textcolor[RGB]{128,0,128}{#1}}$
$\newcommand{\ywch}[2]{\textcolor{mygray}{\sout{#1}}{\textcolor[RGB]{139,0,139}{{#2}}}}$
$\newcommand{\ywrem}[1]{\textcolor{mygray}{\sout{#1}}}$
$\newcommand{\App}[1]{Appendix~\ref{sec:#1}}$
$\newcommand{\app}[1]{App.~\ref{sec:#1}}$
$\newcommand{\Se}[1]{Section~\ref{sec:#1}}$
$\newcommand{\se}[1]{Sect.~\ref{sec:#1}}$
$\newcommand{\Fg}[1]{Figure~\ref{fig:#1}}$
$\newcommand{\fg}[1]{Fig.~\ref{fig:#1}}$
$\newcommand{\fgs}[2]{Figs.~\ref{fig:#1} and \ref{fig:#2}}$
$\newcommand{\tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\Tb}[1]{Table~\ref{tab:#1}}$
$\newcommand{\eq}[1]{equation~(\ref{eq:#1})}$
$\newcommand{\Eq}[1]{Equation~(\ref{eq:#1})}$
$\newcommand{\Eqs}[2]{Equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqs}[2]{equations~(\ref{eq:#1}) and (\ref{eq:#2})}$
$\newcommand{\eqto}[2]{equations~(\ref{eq:#1})--(\ref{eq:#2})}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\ywc}[1]$
$\newcommand{\ywrem}[1]{\xspace}$
$\newcommand{\ywch}[2]{#2}$
$\newcommand{\ywadd}[1]{#1\xspace}$
$\newcommand{\cim}[1]$
$\newcommand{\?}$
$\newcommand{\corem}[1]{\xspace}$
$\newcommand{çc}[1]{\xspace}$
$\newcommand{\coch}[2]{#2}$
$\newcommand{\coadd}[1]{#1\xspace}$
$\newcommand{\md}$
$\newcommand{\mdd}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Dust and Water in V883 Ori: Relics of a Retreating Snowline

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.27468-b31b1b.svg)](https://arxiv.org/abs/2510.27468)<mark>Appeared on: 2025-11-03</mark> -  _Accepted for publication in A&A Letter_

</div>
<div id="authors">

<mark>Y. Wang</mark>, et al.

</div>
<div id="abstract">

**Abstract:** V883 Ori is an FU-Orionis-type outburst system characterized by a shoulder at 50-70 au in its ALMA band 6 and 7 intensity profiles. Previously, this feature was attributed to dust pile-up from pebble disintegration at the water snowline. However, recent multi-wavelength observations show continuity in the spectral index across the expected snowline region, disfavoring abrupt changes in grain properties. Moreover, extended water emission is detected beyond 80 au, pointing to a snowline further out. This Letter aims to explain both features with a model in which the snowline is receding.    We construct a 2D disk model that solves the cooling and subsequent vapor recondensation during the post-outburst dimming phase.    Our results show that both the intensity shoulder and the extended water emission are natural relics of a retreating snowline: the shoulder arises from excess surface density generated by vapor recondensation at the moving condensation front, while the outer water vapor reservoir persists due to the long recondensation timescales of $10^{2}-10^{3}$ yr at the disk atmosphere.    As V883 Ori continues to fade, we predict that the intensity shoulder will migrate inward by an observationally significant amount of 10 au over about $\md{25}$ years.

</div>

<div id="div_fig1">

<img src="tmp_2510.27468/./fig/dimming_curve_revise.png" alt="Fig1" width="100%"/>

**Figure 1. -** Evolution of the bolometric luminosity of V883 Ori during the outburst dimming phase. The symbols denote measurements from different studies, $\md${with horizontal bars showing the time range of adopted data}. The blue band represents the adopted luminosity curve, which carries an uncertainty of ${\sim}40 \mathrm{yrs}$ due to the uncertainties in estimating $L_{\mathrm{bol}}$. (*fig:dimming_curve*)

</div>
<div id="div_fig2">

<img src="tmp_2510.27468/./fig/sigma_time_revise_45_85.png" alt="Fig2" width="100%"/>

**Figure 2. -** Time evolution of the surface density profiles of the best-fit model. Silicate surface density remains constant with time. The thick line denotes the best-fit $t^{\prime}=121$ yr. (*fig:sigma_time*)

</div>
<div id="div_fig3">

<img src="tmp_2510.27468/./fig/bestfit_intens_revise_45_85.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison of the synthetic intensity profiles with ALMA continuum. Crosses show ALMA data adopted from \citet{HougeEtal2024}, spaced according to the beam sizes; shaded regions indicate the rms error. Solid lines represent the best-fit retreating snowline model, while dotted lines correspond to the best-fit static model. Enhanced emission towards the inner regions arises likely from intense viscous heating \citep{AlarconEtal2024}, which is not accounted for in the model. (*fig:obv_compare*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.27468"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tg}[1]{\textcolor{violet}{#1}}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$\/: Systematic uncertainties from the halo mass conversion on galaxy cluster number count data analyses$\thanks{This paper is published on     behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.27505-b31b1b.svg)](https://arxiv.org/abs/2510.27505)<mark>Appeared on: 2025-11-03</mark> -  _20 pages, 11 figures, submitted to A&A_

</div>
<div id="authors">

T. Gayoux, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The large catalogues of galaxy clusters expected from the $_ Euclid_$ survey will enable cosmological analyses of cluster number counts that require accurate cosmological model predictions. One possibility is to use parametric fits calibrated against $N$ -body simulations, that capture the cosmological parameter dependence of the halo mass function. Several studies have shown that this can be obtained through a calibration against haloes with spherical masses defined at the virial overdensity. In contrast, if different mass definitions are used for the HMF and the scaling relation, a mapping between them is required. Here, we investigate the impact of such a mapping on the cosmological parameter constraints inferred from galaxy cluster number counts. Using synthetic data from $N$ -body simulations, we show that the standard approach, which relies on assuming a concentration-mass relation, can introduce significant systematic bias. In particular, depending on the mass definition and the relation assumed, this can lead to biased constraints at more than 2 $\sigma$ level. In contrast, we find that in all the cases we have considered, the mass conversion based on the halo sparsity statistics result in a systematic bias smaller than the statistical error.

</div>

<div id="div_fig1">

<img src="tmp_2510.27505/./figures/Flagship_all_200c_zcuts.png" alt="Fig6.1" width="50%"/><img src="tmp_2510.27505/./figures/Flagship_all_500c_zcuts.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Mean and standard deviation of the marginalised constraints on $\Omega_{\rm m}$(_ top panels_) and $\sigma_8$(_ bottom panels_) inferred from the analysis of Flagship data at $M_{200}$(_ left panels_) and $M_{500}$(_ right panels_). In each _ panel_ the left-hand (right-hand) side corresponds to the low (high) redshift cut. These have been obtained using the NPS mass conversion (magenta stars), the PD (circles) and PS (squares) approaches for different concentration-mass relations as in Fig. \ref{fig:summary_plot_cM_omegam}. The empty (filled) symbols correspond to the constraints inferred assuming the _ Euclid_/Uchuu-HMF (_ Euclid_-HMF) mapped to the target mass definition of the Flagship datasets. The vertical lines show the fiducial value of $\Omega_{\rm m}$ and $\sigma_8$ respectively. (*fig:summary_plot_cM_omegam_flagship*)

</div>
<div id="div_fig2">

<img src="tmp_2510.27505/./figures/euclid_all_200c.png" alt="Fig5.1" width="50%"/><img src="tmp_2510.27505/./figures/euclid_all_500c.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Mean and standard deviation of the marginalised constraints on $\Omega_{\rm m}$(_ top panels_) and $\sigma_8$(_ bottom panels_) inferred from the analysis of Uchuu data at $M_{200}$(_ left panels_) and $M_{500}$(_ right panels_). In each _ panel_ the left-hand (right-hand) side corresponds to the low (high) mass cuts. These have been obtained by applying the mass conversion to the _ Euclid_/Uchuu-HMF calibrated at $M_{\rm vir}$ using the NPS (magenta star points) the PD (filled circles) and PS (filled squares) mass conversion approaches. In the PD and PS cases we have assumed concentration-mass relation from the Uchuu dataset \citetalias{Ishiyama_2021}(olive green) and the relations from \citetalias{2014MNRAS.441.3359D}(cyan), \citetalias{2013ApJ...766...32B}(dark blue), \citetalias{2011ApJ...740..102K}(violet) and \citetalias{2008MNRAS.390L..64D}(dark green). The vertical lines shows the fiducial values of $\Omega_{\rm m}$ and $\sigma_8$ respectively. (*fig:summary_plot_cM_omegam*)

</div>
<div id="div_fig3">

<img src="tmp_2510.27505/./figures/side_by_side_comparison_vir_Flagship.png" alt="Fig1" width="100%"/>

**Figure 1. -** _ Top panel_: number counts of haloes detected at $M_{\rm vir}$ from the Flagship light-cone catalogue (green solid line) with $M_{\rm vir}\ge 10^{14} {\rm M}_{\odot}h^{-1}$ over a sky area of ${\pi}/{2}$\si{\steradian} in bins of size $\Delta{z}=0.1$ against the predictions from the _ Euclid_/Uchuu-HMF (orange solid line) and the _ Euclid_-HMF (brown solid line). _ Bottom panel_: relative difference with respect to the Flagship number counts. The shaded area correspond to the Poisson noise. (*fig:flagship_ncount_mvir*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.27505"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Multi-band infrared imaging reveals dusty spiral arcs around the binary B[e] star $\object{$3$ Puppis}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.27591-b31b1b.svg)](https://arxiv.org/abs/2510.27591)<mark>Appeared on: 2025-11-03</mark> -  _19 pages, 11 figures, 11 tables_

</div>
<div id="authors">

M. Abello, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** $\object{$3$ Puppis}$ stands out as the brightest star known exhibiting the B [ e ] phenomenon. Although recent studies have classified this A-type star within the supergiant group, the influence of its binary nature on the circumstellar environment (CE) remains difficult to model and interpret. To resolve the dusty regions of $\object{$3$ Puppis}$ at angular scales of 5--10 milliarcseconds (mas), we conducted high-angular-resolution interferometric observations with the mid-infrared beam combiner VLTI/MATISSE across the $3$ -- $12 \mu$ m wavelength range. Since the $(u,v)$ coverage is sufficient to perform image reconstruction, we present an innovative statistical interferometric imaging technique based on the \texttt{MiRA} software to produce averaged images. Applied to MATISSE data, this systematic approach enables the selection of an optimal set of reconstructions, improving the robustness and fidelity of the recovered features. We also apply \texttt{SPARCO} , an independent imaging tool well suited to systems with a bright central source surrounded by a fainter and extended CE. The images obtained with both tools in the L, M, and N spectral bands show good agreement and clearly reveal an asymmetric elongated structure located at $\sim17$ mas ( $\sim10$ au at 631 pc) to the southeast of the image centre, with a density contrast of $\sim20\%$ . A second asymmetry to the northwest and a skewed inner rim are also detected. Simple geometric modelling, inspired by the reconstructed images, provides quantitative constraints on the morphology, position, and flux contribution of the CE and its asymmetries. Our final MATISSE images are consistent with previous VLTI interferometric observations while revealing a more complex CE with large-scale clumps in the southeast and northwest disc regions of $\object{$3$ Puppis}$ . Finally the hydrodynamic simulation concludes that the tidal spiral wake perturbations driven by the central binary, dynamically excited at Lindblad resonances within the circumbinary disc, provide the best interpretation for the radial extent and curvature of the elongated structures observed across all bands.

</div>

<div id="div_fig1">

<img src="tmp_2510.27591/./figures/Results/MYTHRA_PSF_SPRACO.png" alt="Fig11" width="100%"/>

**Figure 11. -** Final image reconstructions of $3$ Puppis in the mid-infrared using \texttt{MiRA}, \texttt{MYTHRA}, and \texttt{SPARCO} imaging tools. The field of view for each image is (60 mas $\times$ 60 mas) with North up and East left. The blue dashed ellipse indicates the best-fit inner rim of the dusty disc derived from VLTI/AMBER data, while the blue cross marks the southeastern elongated clump best-fit position, denoted $R_\mathrm{ref}$, from MATISSE L-band geometric modelling (at a distance of $16.71 \pm 0.03$ mas from the image centre). Brightness colour scale is normalised to peak intensity (i.e. maximum pixel value). _K-band image:_ The left-most column shows the images obtained with VLTI/AMBER data by [Millour, Meilland and Chesneau (2011)](). In the first row the median \texttt{MiRA} image is displayed, and the last two rows present the identical convolved median \texttt{MiRA} image with the $\lambda/2B_\mathrm{max}$ PSF subtraction applied. _L-M-N-bands images:_ The remaining three columns display the images obtained with VLTI/MATISSE. On the first row the resulting \texttt{MYTHRA} averaged image is shown. The second row shows the convolved image of \texttt{MYTHRA} with the $\lambda/2B_\mathrm{max}$ PSF subtraction applied. The last row gives the resulting \texttt{SPARCO} image, convolved with the interferometric beam. (*fig:images*)

</div>
<div id="div_fig2">

<img src="tmp_2510.27591/./figures/Observations/uvplot_lPup_meters_obsdate.png" alt="Fig1" width="100%"/>

**Figure 1. -** Coverage of the ($u,v$)-plane obtained from the VLTI/MATISSE observations of $\object${$3$ Puppis}. The legend in the lower-right corner indicates the colours corresponding to the four AT configurations used: Small (blue), Medium (orange), Large (green), and Extended (red). A schematic map of the VLTI baselines, using the same colour code, is shown in the upper-right corner. East corresponds to increasing $x$-axis values, and North to increasing $y$-axis values. (*fig:uv*)

</div>
<div id="div_fig3">

<img src="tmp_2510.27591/./figures/Observations/lPup_LMN_data.png" alt="Fig9" width="100%"/>

**Figure 9. -** VLTI/MATISSE squared visibility data, denoted V$^2$, in logarithmic scale (_top plot_) and closure phase data, denoted CP, (_bottom plot_) of $\object${$3$ Puppis} in the L- (blue), M- (green), and N- (red) bands, plotted as a function of spatial frequency B/$\lambda$. The vertical dashed lines indicate the approximate positions of the first visibility nulls in the three bands with the same colour code. The horizontal dashed line marks the visibility plateau at long baselines in the L-band, which reflects the relative flux contribution from the unresolved central source. (*fig:dataLMN*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.27591"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

115  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
